# Set Client Margin 

In [1057]:
import pandas as pd
import numpy as np
import math

#For UPS client freight & accessorial discounts, write "YES"/"NO" for rounding/not rounding to 2 decimal places.
Round_Discount = "NO" #default = no

In [1058]:
from IPython.display import HTML

# Create a styled HTML string
styled_html = HTML("""
<p style="font-family: 'Arial'; font-size: 20px; color: red; background-color: yellow;">
    Edit Location 1 of 8
</p>
""")

# Display the styled HTML in the notebook
display(styled_html)


In [1059]:
#Set UPS margin for the client
Margin_on_Min_GRND = 0.0  #----------------------Edit
Margin_on_Min_AIR = 0.0  #-----------------------Edit

Margin_on_UPS_Parcel_Freight_GRND = 0.0 #--------Edit
Margin_on_UPS_Parcel_Freight_AIR = 0.0 #---------Edit

Margin_on_Value_Add_GRND = 0.2 #-----------------Edit
Margin_on_Value_Add_AIR = 0.2 #------------------Edit

#Set DHL, USPS, and MI margin for the client
Margin_on_DHL = 0.0 #----------------------------Edit
Margin_on_USPS = 0.0 #---------------------------Edit
Margin_on_MI = 0.0 #-----------------------------Edit

# Carrier Dim Factor 2023

In [1060]:
#Dim factor 2023
dim_factor_ups = 280
dim_factor_ups_srpt = 139
dim_factor_dhl = 166
dim_factor_usps = 166
dim_factor_mi = 166

# Set the Year based on PLD 
This quote can only apply to 1 year at one time. 

In [1061]:
nqd_dhl_date_1 = '2023-01-01'
nqd_dhl_date_2 = '2023-09-30'
nqd_dhl_date_3 = '2023-10-01'
nqd_dhl_date_4 = '2023-12-31'
nqd_dhl_date_1 = pd.to_datetime(nqd_dhl_date_1)
nqd_dhl_date_2 = pd.to_datetime(nqd_dhl_date_2)
nqd_dhl_date_3 = pd.to_datetime(nqd_dhl_date_3)
nqd_dhl_date_4 = pd.to_datetime(nqd_dhl_date_4)

In [947]:
#OB_STATE = pd.read_csv('C:\\Users\\sliu\\BD\\re-rate_model\\OB_STATE.csv')

##### Get Zone from facility doc
##### When the distribution is all over the U.S., we use TN Facility. Needs business input. Make sure the facility matches 
##### between dhl rates and Zone finder.

#xls = pd.ExcelFile('zipOutbound.xlsx')

##### Load Outbound card

#OB_MA = pd.read_excel(xls, '02038_Outbound') # 5114358_dhl_sm_parcel_Franklin
#OB_NJ = pd.read_excel(xls, '08873_Outbound') # 5122444_dhl_sm_parcel_Somerset
#OB_MD = pd.read_excel(xls, '21226_Outbound') # 5123283_dhl_sm_parcel_Curtis Bay, GBM
#OB_TN = pd.read_excel(xls, '38141_Outbound') # 5122893_dhl_sm_parcell Memphis
#OB_MS = pd.read_excel(xls, '38654_Outbound') # 5122723_dhl_sm_parcel_8150 Nail Olive Branch, MS3/5122739_dhl_sm_parcel_Nail rd Olive branch, MS2
#OB_TX = pd.read_excel(xls, '75041_Outbound') # 5122855_dhl_sm_parcel_Garland TX
#OB_CA = pd.read_excel(xls, '90640_Outbound') # none

#OB_02324 = pd.read_excel(xls, '02324_Outbound')
#OB_06096 = pd.read_excel(xls, '06096_Outbound')
#OB_18105 = pd.read_excel(xls, '18105_Outbound')
#OB_21240 = pd.read_excel(xls, '21240_Outbound')
#OB_43004 = pd.read_excel(xls, '43004_Outbound')

#OB_MA_1 = OB_MA.copy()
#OB_MA_1 = OB_MA_1.drop('Origin', axis=1)
#OB_MA_1['Origin'] = '2048'

#OB_STATE = pd.concat([OB_MA, OB_NJ, OB_MD, OB_TN, OB_MS, OB_TX, OB_CA, OB_MA_1, OB_02324, OB_06096, OB_18105, OB_21240, OB_43004])

##### n/a - 5122890_dhl_sm_parcel_Oofos, Oofos is (or will be in Byhalia, MS)
##### n/a - 5123280_dhl_sm_parcel_MAN, 02048
##### n/a - 5123282_dhl_sm_parcel Bridgewater, MA5
##### 90640 is Montebello, CA (I don't see that here)   

#OB_STATE.to_csv('OB_STATE.csv')


# ======Model Structure======

Source tables:
1. carrierRateCard_2023: published freight rate card (insert newer version once a year)

   1-1. UPSAccessorial_2023: UPS accessorial rate card. (insert newer version once a year)


2. areaSurchargeZipsUs_2023: a table to assign the type of DAS Category (insert newer version once a year)

3. FSC: fuel surcharge rate (insert a new number once a week before automation)

4. zip_Outbound: after decided the origin facility, get zone from destination zip 

5. ZiptoState: get state from destination zip to decide the origin facility
    
6. PLD: for rerate

Structure:

1 - a. Create Freight Rate Card (tbl_PublishedRates)  
1 - b. Create Freight Rate Card (tbl_BarrettRates)  
1 - c. Create Freight Rate Card (tbl_clientRates)

2 - a. Create UPS accessorial rates  
2 - b. Prepare all other rates for rerate

3 - a. Prepare PLD for rerate: Load, Clean, and Format PLD  
3 - b. Rerate Begin  
3 - c. Output and analysis




| Carrier | Service | Residential | WeightType | RateType   |
|:--------|:---------|:-------------|:------------|:------------|
| UPS     | REDE (UPS NDA Early) / RED (UPS NDA) / REDS (UPS NDA Saver) / 2DAM (UPS 2DA A.M.) / BLUE (UPS 2DA) / ORNG (UPS 3DA) / SRPT (UPS Surepost) | both         | lb         | Published   |
| UPS     | GRND (UPS Ground Commercial)                                                                                                              | commercial   | lb         | Published   |
| UPS     | GRES (UPS Ground Residential)                                                                                                             | residential  | lb         | Published   |
| UPS     | SRPT<1 (UPS Surepost 1#>)                                                                                                                 | both         | oz         | Published   |
| DHL     | DHLG (DHL SmartMail Parcel Plus Ground 1-25)/DHLE (DHL SmartMail Parcel Plus Expedited 1-25) | both | lb | Barrett |
| DHL     | DHLG<1 (DHL SmartMail Parcel Ground < 1lb)/DHLE<1 (DHL SmartMail Parcel Expedited  < 1lb)/DHLEM (DHL SmartMail Parcel Expedited Max) | both | lb | Barrett |
| USPS    | USPSAG (USPS Auctane GRND) | both | lb | Barrett |
| USPS    | USPSAP (USPS Auctane PM)| both | lb | Barrett |
| MI      | MIPLE (MI Parcel Select Lightweight Expedited)|  both | oz | Barrett |
| MI      | MIPH (MI Parcel Select Heavyweight)|  both | lb | Barrett |



# Step 1 - a & 1 - b
*rate card: Add UPS published rates, DHL Barrett rates, and USPS Barrett rates. Needs to renew once a year. Can add more carriers later.

In [1062]:
# Read carrierRateCard_2023 and remake the tables showing below columns.
# SERVICE_CODE, SERVICE_PLAINTEXT, ZONE, WEIGHT_OZ, WEIGHT_LB, PUBLISHED_RATES, VERSION, START_DATE, END_DATE
xls = pd.ExcelFile('carrierRateCard_2023.xlsx')
# Load UPS rate card
df1 = pd.read_excel(xls, 'REDE')
df2 = pd.read_excel(xls, 'RED')
df3 = pd.read_excel(xls, 'REDS')
df4 = pd.read_excel(xls, '2DAM')
df5 = pd.read_excel(xls, 'BLUE')
df6 = pd.read_excel(xls, 'ORNG')
df7 = pd.read_excel(xls, 'GRND')
df8 = pd.read_excel(xls, 'GRES')
df9 = pd.read_excel(xls, 'SRPT<1')
df10 = pd.read_excel(xls, 'SRPT')

# Load USPS rate card
df16 = pd.read_excel(xls, 'USPSAG_CPP')
df17 = pd.read_excel(xls, 'USPSAP_CPP')
df20 = pd.read_excel(xls, 'USPSAG_CPPCI')
df21 = pd.read_excel(xls, 'USPSAP_CPPCI')
df22 = pd.read_excel(xls, 'USPSAG_AUCT')
df23 = pd.read_excel(xls, 'USPSAP_AUCT')
df24 = pd.read_excel(xls, 'USPSAG_AUCTCI')
df25 = pd.read_excel(xls, 'USPSAP_AUCTCI')

# Load MI rate card
df18 = pd.read_excel(xls, 'MIPLE')
df19 = pd.read_excel(xls, 'MIPH')

# Load DHL rate card
# 5114358_dhl_sm_parcel_Franklin
df11_1 = pd.read_excel(xls, 'DHLG_5114358')
df12_2 = pd.read_excel(xls, 'DHLG<1_5114358')
df13_3 = pd.read_excel(xls, 'DHLE_5114358')
df14_4 = pd.read_excel(xls, 'DHLE<1_5114358')
df15_5 = pd.read_excel(xls, 'DHLEM_5114358')

# 5122444_dhl_sm_parcel_Somerset
df11_6 = pd.read_excel(xls, 'DHLG_5122444')
df12_7 = pd.read_excel(xls, 'DHLG<1_5122444')
df13_8 = pd.read_excel(xls, 'DHLE_5122444')
df14_9 = pd.read_excel(xls, 'DHLE<1_5122444')
#df15_10 = pd.read_excel(xls, 'DHLEM_5122444')

# 5122723_dhl_sm_parcel_8150 Nail Olive Branch
df11_11 = pd.read_excel(xls, 'DHLG_5122723')
df12_12 = pd.read_excel(xls, 'DHLG<1_5122723')
df13_13 = pd.read_excel(xls, 'DHLE_5122723')
df14_14 = pd.read_excel(xls, 'DHLE<1_5122723')
df15_15 = pd.read_excel(xls, 'DHLEM_5122723')

# 5122739_dhl_sm_parcel_Nail rd Olive branch
df11_16 = pd.read_excel(xls, 'DHLG_5122739')
df12_17 = pd.read_excel(xls, 'DHLG<1_5122739')
df13_18 = pd.read_excel(xls, 'DHLE_5122739')
df14_19 = pd.read_excel(xls, 'DHLE<1_5122739')
df15_20 = pd.read_excel(xls, 'DHLEM_5122739')

# 5122855_dhl_sm_parcel_Garland TX
df11_21 = pd.read_excel(xls, 'DHLG_5122855')
df12_22 = pd.read_excel(xls, 'DHLG<1_5122855')
df13_23 = pd.read_excel(xls, 'DHLE_5122855')
df14_24 = pd.read_excel(xls, 'DHLE<1_5122855')
df15_25 = pd.read_excel(xls, 'DHLEM_5122855')

# 5122890_dhl_sm_parcel_Oofos
df11_26 = pd.read_excel(xls, 'DHLG_5122890')
df12_27 = pd.read_excel(xls, 'DHLG<1_5122890')
df13_28 = pd.read_excel(xls, 'DHLE_5122890')
df14_29 = pd.read_excel(xls, 'DHLE<1_5122890')
df15_30 = pd.read_excel(xls, 'DHLEM_5122890')

# 5122893_dhl_sm_parcel Memphis
df11_31 = pd.read_excel(xls, 'DHLG_5122893')
df12_32 = pd.read_excel(xls, 'DHLG<1_5122893')
df13_33 = pd.read_excel(xls, 'DHLE_5122893')
df14_34 = pd.read_excel(xls, 'DHLE<1_5122893')
df15_35 = pd.read_excel(xls, 'DHLEM_5122893')

# 5123280_dhl_sm_parcel_MAN
df11_36 = pd.read_excel(xls, 'DHLG_5123280')
df12_37 = pd.read_excel(xls, 'DHLG<1_5123280')
df13_38 = pd.read_excel(xls, 'DHLE_5123280')
df14_39 = pd.read_excel(xls, 'DHLE<1_5123280')
df15_40 = pd.read_excel(xls, 'DHLEM_5123280')

# 5123282_dhl_sm_parcel Bridgewater
df11_41 = pd.read_excel(xls, 'DHLG_5123282')
df12_42 = pd.read_excel(xls, 'DHLG<1_5123282')
df13_43 = pd.read_excel(xls, 'DHLE_5123282')
df14_44 = pd.read_excel(xls, 'DHLE<1_5123282')
df15_45 = pd.read_excel(xls, 'DHLEM_5123282')

# 5123283_dhl_sm_parcel_Curtis Bay
df11_46 = pd.read_excel(xls, 'DHLG_5123283')
df12_47 = pd.read_excel(xls, 'DHLG<1_5123283')
df13_48 = pd.read_excel(xls, 'DHLE_5123283')
df14_49 = pd.read_excel(xls, 'DHLE<1_5123283')
df15_50 = pd.read_excel(xls, 'DHLEM_5123283')


# 5123283_dhl_sm_parcel_Curtis Bay
#df11_51 = pd.read_excel(xls, 'DHLG_5123556')
#df12_52 = pd.read_excel(xls, 'DHLG<1_5123556')
#df13_53 = pd.read_excel(xls, 'DHLE_5123556')
#df14_54 = pd.read_excel(xls, 'DHLE<1_5123556')
#df15_55 = pd.read_excel(xls, 'DHLEM_5123556')



In [1063]:
# Re-arrange the tables
# REDE, Barrett Rates = Published Rates. 
# made sure the tables are completely loaded into the dataframe by checking df1.shape

df1 = df1.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df2 = df2.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df3 = df3.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df4 = df4.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df5 = df5.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df6 = df6.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df7 = df7.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df8 = df8.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df9 = df9.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='PUBLISHED_RATES')
df10 = df10.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')


# put cpp price as Barrett Rate to ease the calculation
df16 = df16.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df17 = df17.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df20 = df20.set_index('Zones').unstack().rename_axis(('ZONE','CUBIC_FT')).reset_index(name='BARRETT_RATES')
df21 = df21.set_index('Zones').unstack().rename_axis(('ZONE','CUBIC_FT')).reset_index(name='BARRETT_RATES')
df22 = df22.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df23 = df23.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df24 = df24.set_index('Zones').unstack().rename_axis(('ZONE','CUBIC_FT')).reset_index(name='BARRETT_RATES')
df25 = df25.set_index('Zones').unstack().rename_axis(('ZONE','CUBIC_FT')).reset_index(name='BARRETT_RATES')

df18 = df18.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df19 = df19.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')

df11_1 = df11_1.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_2 = df12_2.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_3 = df13_3.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_4 = df14_4.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_5 = df15_5.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_6 = df11_6.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_7 = df12_7.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_8 = df13_8.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_9 = df14_9.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
#df15_10 = df15_10.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_11 = df11_11.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_12 = df12_12.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_13 = df13_13.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_14 = df14_14.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_15 = df15_15.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_16 = df11_16.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_17 = df12_17.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_18 = df13_18.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_19 = df14_19.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_20 = df15_20.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_21 = df11_21.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_22 = df12_22.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_23 = df13_23.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_24 = df14_24.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_25 = df15_25.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_26 = df11_26.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_27 = df12_27.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_28 = df13_28.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_29 = df14_29.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_30 = df15_30.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_31 = df11_31.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_32 = df12_32.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_33 = df13_33.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_34 = df14_34.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_35 = df15_35.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_36 = df11_36.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_37 = df12_37.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_38 = df13_38.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_39 = df14_39.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_40 = df15_40.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_41 = df11_41.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_42 = df12_42.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_43 = df13_43.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_44 = df14_44.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_45 = df15_45.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_46 = df11_46.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_47 = df12_47.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_48 = df13_48.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_49 = df14_49.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_50 = df15_50.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

#df11_51 = df11_51.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
#df12_52 = df12_52.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
#df13_53 = df13_53.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
#df14_54 = df14_54.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
#df15_55 = df15_55.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

# Make sure the dataframes have both weight in lb and oz columns
df1['WEIGHT_OZ'] = df1['WEIGHT_LB']*16
df2['WEIGHT_OZ'] = df2['WEIGHT_LB']*16
df3['WEIGHT_OZ'] = df3['WEIGHT_LB']*16
df4['WEIGHT_OZ'] = df4['WEIGHT_LB']*16
df5['WEIGHT_OZ'] = df5['WEIGHT_LB']*16
df6['WEIGHT_OZ'] = df6['WEIGHT_LB']*16
df7['WEIGHT_OZ'] = df7['WEIGHT_LB']*16
df8['WEIGHT_OZ'] = df8['WEIGHT_LB']*16
df9['WEIGHT_LB'] = df9['WEIGHT_OZ']/16
df10['WEIGHT_OZ'] = df10['WEIGHT_LB']*16

df16['WEIGHT_OZ'] = df16['WEIGHT_LB']*16
df17['WEIGHT_OZ'] = df17['WEIGHT_LB']*16

df22['WEIGHT_OZ'] = df22['WEIGHT_LB']*16
df23['WEIGHT_OZ'] = df23['WEIGHT_LB']*16

df18['WEIGHT_LB'] = df18['WEIGHT_OZ']/16
df19['WEIGHT_OZ'] = df19['WEIGHT_LB']*16

df11_1['WEIGHT_OZ'] = df11_1['WEIGHT_LB']*16
df12_2['WEIGHT_LB'] = df12_2['WEIGHT_OZ']/16
df13_3['WEIGHT_OZ'] = df13_3['WEIGHT_LB']*16
df14_4['WEIGHT_LB'] = df14_4['WEIGHT_OZ']/16
df15_5['WEIGHT_LB'] = df15_5['WEIGHT_OZ']/16

df11_6['WEIGHT_OZ'] = df11_6['WEIGHT_LB']*16
df12_7['WEIGHT_LB'] = df12_7['WEIGHT_OZ']/16
df13_8['WEIGHT_OZ'] = df13_8['WEIGHT_LB']*16
df14_9['WEIGHT_LB'] = df14_9['WEIGHT_OZ']/16
#df15_10['WEIGHT_LB'] = df15_10['WEIGHT_OZ']/16

df11_11['WEIGHT_OZ'] = df11_11['WEIGHT_LB']*16
df12_12['WEIGHT_LB'] = df12_12['WEIGHT_OZ']/16
df13_13['WEIGHT_OZ'] = df13_13['WEIGHT_LB']*16
df14_14['WEIGHT_LB'] = df14_14['WEIGHT_OZ']/16
df15_15['WEIGHT_LB'] = df15_15['WEIGHT_OZ']/16

df11_16['WEIGHT_OZ'] = df11_16['WEIGHT_LB']*16
df12_17['WEIGHT_LB'] = df12_17['WEIGHT_OZ']/16
df13_18['WEIGHT_OZ'] = df13_18['WEIGHT_LB']*16
df14_19['WEIGHT_LB'] = df14_19['WEIGHT_OZ']/16
df15_20['WEIGHT_LB'] = df15_20['WEIGHT_OZ']/16

df11_21['WEIGHT_OZ'] = df11_21['WEIGHT_LB']*16
df12_22['WEIGHT_LB'] = df12_22['WEIGHT_OZ']/16
df13_23['WEIGHT_OZ'] = df13_23['WEIGHT_LB']*16
df14_24['WEIGHT_LB'] = df14_24['WEIGHT_OZ']/16
df15_25['WEIGHT_LB'] = df15_25['WEIGHT_OZ']/16

df11_26['WEIGHT_OZ'] = df11_26['WEIGHT_LB']*16
df12_27['WEIGHT_LB'] = df12_27['WEIGHT_OZ']/16
df13_28['WEIGHT_OZ'] = df13_28['WEIGHT_LB']*16
df14_29['WEIGHT_LB'] = df14_29['WEIGHT_OZ']/16
df15_30['WEIGHT_LB'] = df15_30['WEIGHT_OZ']/16

df11_31['WEIGHT_OZ'] = df11_31['WEIGHT_LB']*16
df12_32['WEIGHT_LB'] = df12_32['WEIGHT_OZ']/16
df13_33['WEIGHT_OZ'] = df13_33['WEIGHT_LB']*16
df14_34['WEIGHT_LB'] = df14_34['WEIGHT_OZ']/16
df15_35['WEIGHT_LB'] = df15_35['WEIGHT_OZ']/16

df11_36['WEIGHT_OZ'] = df11_36['WEIGHT_LB']*16
df12_37['WEIGHT_LB'] = df12_37['WEIGHT_OZ']/16
df13_38['WEIGHT_OZ'] = df13_38['WEIGHT_LB']*16
df14_39['WEIGHT_LB'] = df14_39['WEIGHT_OZ']/16
df15_40['WEIGHT_LB'] = df15_40['WEIGHT_OZ']/16

df11_41['WEIGHT_OZ'] = df11_41['WEIGHT_LB']*16
df12_42['WEIGHT_LB'] = df12_42['WEIGHT_OZ']/16
df13_43['WEIGHT_OZ'] = df13_43['WEIGHT_LB']*16
df14_44['WEIGHT_LB'] = df14_44['WEIGHT_OZ']/16
df15_45['WEIGHT_LB'] = df15_45['WEIGHT_OZ']/16

df11_46['WEIGHT_OZ'] = df11_46['WEIGHT_LB']*16
df12_47['WEIGHT_LB'] = df12_47['WEIGHT_OZ']/16
df13_48['WEIGHT_OZ'] = df13_48['WEIGHT_LB']*16
df14_49['WEIGHT_LB'] = df14_49['WEIGHT_OZ']/16
df15_50['WEIGHT_LB'] = df15_50['WEIGHT_OZ']/16


#df11_51['WEIGHT_OZ'] = df11_51['WEIGHT_LB']*16
#df12_52['WEIGHT_LB'] = df12_52['WEIGHT_OZ']/16
#df13_53['WEIGHT_OZ'] = df13_53['WEIGHT_LB']*16
#df14_54['WEIGHT_LB'] = df14_54['WEIGHT_OZ']/16
#df15_55['WEIGHT_LB'] = df15_55['WEIGHT_OZ']/16


# use below if the charts have empty spaces. No need to use here as the source tables are already clean.
# df1 = df1.dropna(axis = 0, how = 'all')
# df1 = df1.dropna(axis = 1, how = 'all')
# df1 = df1.dropna()

# Add Service code and Service columns
df1['SERVICE_PLAINTEXT']='UPS NDA Early'
df2['SERVICE_PLAINTEXT']='UPS NDA'
df3['SERVICE_PLAINTEXT']='UPS NDA Saver'
df4['SERVICE_PLAINTEXT']='UPS 2DA A.M.'
df5['SERVICE_PLAINTEXT']='UPS 2DA'
df6['SERVICE_PLAINTEXT']='UPS 3DA'
df7['SERVICE_PLAINTEXT']='UPS Ground Commercial'
df8['SERVICE_PLAINTEXT']='UPS Ground Residential'
df9['SERVICE_PLAINTEXT']='UPS Surepost 1#>'
df10['SERVICE_PLAINTEXT']='UPS Surepost'

df16['SERVICE_PLAINTEXT']='USPS Ground Cpp'
df17['SERVICE_PLAINTEXT']='USPS Priority Cpp'

df20['SERVICE_PLAINTEXT']='USPS Ground Cpp Cubic Inch'
df21['SERVICE_PLAINTEXT']='USPS Priority Cpp Cubic Inch'

df22['SERVICE_PLAINTEXT']='USPS Ground Auctane'
df23['SERVICE_PLAINTEXT']='USPS Priority Auctane'

df24['SERVICE_PLAINTEXT']='USPS Ground Auctane Cubic Inch'
df25['SERVICE_PLAINTEXT']='USPS Priority Auctane Cubic Inch'

df18['SERVICE_PLAINTEXT']='MI Parcel Select Lightweight Expedited'
df19['SERVICE_PLAINTEXT']='MI Parcel Select Heavyweight'

# Load DHL rate card
# 5114358_dhl_sm_parcel_Franklin
# 5122444_dhl_sm_parcel_Somerset, no DHLEM
# 5122723_dhl_sm_parcel_8150 Nail Olive Branch
# 5122739_dhl_sm_parcel_Nail rd Olive branch
# 5122855_dhl_sm_parcel_Garland TX
# 5122890_dhl_sm_parcel_Oofos
# 5122893_dhl_sm_parcell Memphis
# 5123280_dhl_sm_parcel_MAN
# 5123282_dhl_sm_parcel Bridgewater
# 5123283_dhl_sm_parcel_Curtis Bay

df11_1['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5114358'
df12_2['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5114358'
df13_3['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5114358'
df14_4['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5114358'
df15_5['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5114358'

df11_6['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122444'
df12_7['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122444'
df13_8['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122444'
df14_9['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122444'
#df15_10['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122444'

df11_11['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122723'
df12_12['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122723'
df13_13['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122723'
df14_14['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122723'
df15_15['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122723'

df11_16['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122739'
df12_17['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122739'
df13_18['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122739'
df14_19['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122739'
df15_20['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122739'

df11_21['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122855'
df12_22['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122855'
df13_23['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122855'
df14_24['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122855'
df15_25['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122855'

df11_26['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122890'
df12_27['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122890'
df13_28['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122890'
df14_29['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122890'
df15_30['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122890'

df11_31['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122893'
df12_32['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122893'
df13_33['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122893'
df14_34['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122893'
df15_35['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122893'

df11_36['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5123280'
df12_37['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5123280'
df13_38['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5123280'
df14_39['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5123280'
df15_40['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5123280'

df11_41['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5123282'
df12_42['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5123282'
df13_43['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5123282'
df14_44['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5123282'
df15_45['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5123282'

df11_46['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5123283'
df12_47['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5123283'
df13_48['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5123283'
df14_49['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5123283'
df15_50['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5123283'

#df11_51['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5123556'
#df12_52['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5123556'
#df13_53['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5123556'
#df14_54['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5123556'
#df15_55['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5123556'



df1['SERVICE_CODE']='REDE'
df2['SERVICE_CODE']='RED'
df3['SERVICE_CODE']='REDS'
df4['SERVICE_CODE']='2DAM'
df5['SERVICE_CODE']='BLUE'
df6['SERVICE_CODE']='ORNG'
df7['SERVICE_CODE']='GRND'
df8['SERVICE_CODE']='GRES'
df9['SERVICE_CODE']='SRPT<1'
df10['SERVICE_CODE']='SRPT'                               
                             
df16['SERVICE_CODE']='USPSAG_CPP'
df17['SERVICE_CODE']='USPSAP_CPP'

df20['SERVICE_CODE']='USPSAG_CPPCI'
df21['SERVICE_CODE']='USPSAP_CPPCI'

df22['SERVICE_CODE']='USPSAG_AUCT'
df23['SERVICE_CODE']='USPSAP_AUCT'

df24['SERVICE_CODE']='USPSAG_AUCTCI'
df25['SERVICE_CODE']='USPSAP_AUCTCI'


df18['SERVICE_CODE']='MIPLE'
df19['SERVICE_CODE']='MIPH'

df11_1['SERVICE_CODE']='DHLG_5114358'
df12_2['SERVICE_CODE']='DHLG<1_5114358'
df13_3['SERVICE_CODE']='DHLE_5114358'
df14_4['SERVICE_CODE']='DHLE<1_5114358'
df15_5['SERVICE_CODE']='DHLEM_5114358'

df11_6['SERVICE_CODE']='DHLG_5122444'
df12_7['SERVICE_CODE']='DHLG<1_5122444'
df13_8['SERVICE_CODE']='DHLE_5122444'
df14_9['SERVICE_CODE']='DHLE<1_5122444'
#df15_10['SERVICE_CODE']='DHLEM_5122444'

df11_11['SERVICE_CODE']='DHLG_5122723'
df12_12['SERVICE_CODE']='DHLG<1_5122723'
df13_13['SERVICE_CODE']='DHLE_5122723'
df14_14['SERVICE_CODE']='DHLE<1_5122723'
df15_15['SERVICE_CODE']='DHLEM_5122723'

df11_16['SERVICE_CODE']='DHLG_5122739'
df12_17['SERVICE_CODE']='DHLG<1_5122739'
df13_18['SERVICE_CODE']='DHLE_5122739'
df14_19['SERVICE_CODE']='DHLE<1_5122739'
df15_20['SERVICE_CODE']='DHLEM_5122739'

df11_21['SERVICE_CODE']='DHLG_5122855'
df12_22['SERVICE_CODE']='DHLG<1_5122855'
df13_23['SERVICE_CODE']='DHLE_5122855'
df14_24['SERVICE_CODE']='DHLE<1_5122855'
df15_25['SERVICE_CODE']='DHLEM_5122855'

df11_26['SERVICE_CODE']='DHLG_5122890'
df12_27['SERVICE_CODE']='DHLG<1_5122890'
df13_28['SERVICE_CODE']='DHLE_5122890'
df14_29['SERVICE_CODE']='DHLE<1_5122890'
df15_30['SERVICE_CODE']='DHLEM_5122890'

df11_31['SERVICE_CODE']='DHLG_5122893'
df12_32['SERVICE_CODE']='DHLG<1_5122893'
df13_33['SERVICE_CODE']='DHLE_5122893'
df14_34['SERVICE_CODE']='DHLE<1_5122893'
df15_35['SERVICE_CODE']='DHLEM_5122893'

df11_36['SERVICE_CODE']='DHLG_5123280'
df12_37['SERVICE_CODE']='DHLG<1_5123280'
df13_38['SERVICE_CODE']='DHLE_5123280'
df14_39['SERVICE_CODE']='DHLE<1_5123280'
df15_40['SERVICE_CODE']='DHLEM_5123280'

df11_41['SERVICE_CODE']='DHLG_5123282'
df12_42['SERVICE_CODE']='DHLG<1_5123282'
df13_43['SERVICE_CODE']='DHLE_5123282'
df14_44['SERVICE_CODE']='DHLE<1_5123282'
df15_45['SERVICE_CODE']='DHLEM_5123282'

df11_46['SERVICE_CODE']='DHLG_5123283'
df12_47['SERVICE_CODE']='DHLG<1_5123283'
df13_48['SERVICE_CODE']='DHLE_5123283'
df14_49['SERVICE_CODE']='DHLE<1_5123283'
df15_50['SERVICE_CODE']='DHLEM_5123283'


#df11_51['SERVICE_CODE']='DHLG_5123556'
#df12_52['SERVICE_CODE']='DHLG<1_5123556'
#df13_53['SERVICE_CODE']='DHLE_5123556'
#df14_54['SERVICE_CODE']='DHLE<1_5123556'
#df15_55['SERVICE_CODE']='DHLEM_5123556'


#Concatenate all together after rearranging the columns
df1 = df1.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df2 = df2.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df3 = df3.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df4 = df4.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df5 = df5.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df6 = df6.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df7 = df7.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df8 = df8.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df9 = df9.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df10 = df10.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])

df16 = df16.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df17 = df17.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df20 = df20.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'CUBIC_FT', 'BARRETT_RATES'])
df21 = df21.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'CUBIC_FT', 'BARRETT_RATES'])

df22 = df22.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df23 = df23.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df24 = df24.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'CUBIC_FT', 'BARRETT_RATES'])
df25 = df25.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'CUBIC_FT', 'BARRETT_RATES'])


df18 = df18.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df19 = df19.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_1 = df11_1.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_2 = df12_2.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_3 = df13_3.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_4 = df14_4.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_5 = df15_5.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_6 = df11_6.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_7 = df12_7.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_8 = df13_8.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_9 = df14_9.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
#df15_10 = df15_10.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_11 = df11_11.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_12 = df12_12.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_13 = df13_13.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_14 = df14_14.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_15 = df15_15.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_16 = df11_16.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_17 = df12_17.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_18 = df13_18.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_19 = df14_19.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_20 = df15_20.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_21 = df11_21.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_22 = df12_22.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_23 = df13_23.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_24 = df14_24.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_25 = df15_25.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_26 = df11_26.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_27 = df12_27.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_28 = df13_28.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_29 = df14_29.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_30 = df15_30.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_31 = df11_31.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_32 = df12_32.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_33 = df13_33.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_34 = df14_34.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_35 = df15_35.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_36 = df11_36.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_37 = df12_37.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_38 = df13_38.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_39 = df14_39.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_40 = df15_40.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_41 = df11_41.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_42 = df12_42.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_43 = df13_43.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_44 = df14_44.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_45 = df15_45.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_46 = df11_46.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_47 = df12_47.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_48 = df13_48.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_49 = df14_49.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_50 = df15_50.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

#df11_51 = df11_51.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
#df12_52 = df12_52.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
#df13_53 = df13_53.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
#df14_54 = df14_54.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
#df15_55 = df15_55.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])




tbl_publishedRates = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df16, df17, df20, df21, df22, df23, df24, df25, df18, df19,
                               df11_1, df12_2, df13_3, df14_4, df15_5, df11_6, df12_7, df13_8, df14_9, #df15_10,
                               df11_11, df12_12, df13_13, df14_14, df15_15, df11_16, df12_17, df13_18, df14_19, df15_20, 
                               df11_21, df12_22, df13_23, df14_24, df15_25, df11_26, df12_27, df13_28, df14_29, df15_30, 
                               df11_31, df12_32, df13_33, df14_34, df15_35, df11_36, df12_37, df13_38, df14_39, df15_40, 
                               df11_41, df12_42, df13_43, df14_44, df15_45, df11_46, df12_47, df13_48, df14_49, df15_50], axis=0)



In [1064]:
# Add the version column
tbl_publishedRates['VERSION'] ='2023'
tbl_publishedRates['START_DATE'] ='2023/01/01'
tbl_publishedRates['END_DATE'] ='2023/12/31'

# Unify the Zone values
def get_portion(value):
    if isinstance(value, int):
        return str(value)  # convert integer to string
    if 'Zone' in value:
        return value[5:]
    elif 'US' in value:
        return value[3:]
    
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE'].apply(get_portion)

# Dictionary to define the replacements
replacements = {
    '20': '', '30': '', '102': '2', '103': '3', '104': '4', '105': '5', '106': '6',
    '107': '7', '108': '8', '202': '2', '203': '3', '204': '4', '205': '5', '206': '6',
    '207': '7', '208': '8', '209': '9', '302': '2', '303': '3', '304': '4', '305': '5', '306': '6',
    '307': '7', '308': '8', '124': '9', '125': '10', '126': '11', '224': '9',
    '225': '10', '226': '11', '44': '9', '45': '10', '46': '11', '03': '3', '04': '4',
    '05': '5', '06': '6', '07': '7', '08': '8', '09': '9'
}

# Function to replace based on the replacements dictionary
def apply_replacements(zone):
    if zone is None:
        return zone  # Or return a default value like '' (empty string) if that's appropriate for your use case
    for old, new in replacements.items():
        zone = zone.replace(old, new)
    return zone


# Apply the replacement function
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].apply(apply_replacements)

# Update 'ZONE' with 'ZONE_new' values where 'ZONE_new' is not None or NaN; otherwise, keep 'ZONE' values
tbl_publishedRates['ZONE'] = tbl_publishedRates['ZONE_new'].combine_first(tbl_publishedRates['ZONE'])

# Drop the 'ZONE_new' column as it's no longer needed
tbl_publishedRates = tbl_publishedRates.drop('ZONE_new', axis=1)


#adjust the index and finish creating this table.
tbl_publishedRates.reset_index(drop=True, inplace=True)

# tbl_publishedRates.head()
# tbl_publishedRates.to_csv('test.csv')

In [1065]:

styled_html = HTML("""
<p style="font-family: 'Arial'; font-size: 20px; color: red; background-color: yellow;">
    Edit Location 2 of 8 (optional)
</p>
""")

display(styled_html)

In [1066]:
#Run Code 1 if need Auctane, Run Code 2 if need CPP. Default Code 1 Auctane. 
# Code 1
# Keep rows where 'ServiceCode' is not in ['cpp1', 'cpp2', 'cpp3']
tbl_publishedRates = tbl_publishedRates[~tbl_publishedRates['SERVICE_CODE'].isin(['USPSAG_CPP', 'USPSAP_CPP', 'USPSAG_CPPCI', 'USPSAP_CPPCI'])] # Edit, CPP


# 2. Replacing values
# Define a mapping of old values to new values
replacement_dict = {
    'USPSAG_AUCT': 'USPSAG',  # Edit, CPP
    'USPSAP_AUCT': 'USPSAP',  # Edit, CPP
    'USPSAG_AUCTCI': 'USPSAG_CI',  # Edit, CPP
    'USPSAP_AUCTCI': 'USPSAP_CI' # Edit, CPP
    
}

# Replace the values in 'ServiceCode' column
tbl_publishedRates['SERVICE_CODE'] = tbl_publishedRates['SERVICE_CODE'].replace(replacement_dict)


In [1067]:
# Code 2
#tbl_publishedRates = tbl_publishedRates[~tbl_publishedRates['SERVICE_CODE'].isin(['USPSAG_AUCT', 'USPSAP_AUCT', 'USPSAG_AUCTCI', 'USPSAP_AUCTCI'])]

#replacement_dict = {
#    'USPSAG_CPP': 'USPSAG',
#    'USPSAP_CPP': 'USPSAP',
#    'USPSAG_CPPCI': 'USPSAG_CI',
#    'USPSAP_CPPCI': 'USPSAP_CI'
    
#}

#tbl_publishedRates['SERVICE_CODE'] = tbl_publishedRates['SERVICE_CODE'].replace(replacement_dict)
  

# Step 1 - b. Create Freight Rate Card (tbl_BarrettRates)
*Add ups BarrettRates to tbl_publishedRates

In [1068]:
#The result is BarrettRate when margin = 0.
#For UPS client freight & accessorial discounts, write "YES"/"NO" for rounding/not rounding to 2 decimal places.
Round_Discount = "NO"

#Set UPS margin for the client
Margin_on_Min_GRND_BR = 0
Margin_on_Min_AIR_BR = 0

Margin_on_UPS_Parcel_Freight_GRND_BR = 0
Margin_on_UPS_Parcel_Freight_AIR_BR = 0

Margin_on_Value_Add_GRND_BR = 0
Margin_on_Value_Add_AIR_BR = 0


# UPSAccessorial_2023 has 2 tabs, UPS_Rates and UPS_Accessorial. UPS_Rates is used to create Barrett Rates
xls = pd.ExcelFile('UPSAccessorial_2023.xlsx')
UPS_Rates = pd.read_excel(xls, 'UPS_Rates')

# This is the value of the left side of "Rates" tab from "Customer Rate Contract Cards 2023" doc
# UPS_Rates


#add 2 columns to this table with min and max weight
def f(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 5
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 10
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 20
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 30
    elif (col['WEIGHT_BREAK'] == '31+lb'):
        return 999
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-16oz'):
        return 16
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-3lb'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-6lb'):
        return 6
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '7-8lb'):
        return 8
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '9lb'):
        return 9  

def g(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 1
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 6
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 11
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 21
    elif col['WEIGHT_BREAK'] == '31+lb':
        return 31
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-16oz'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-3lb'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-6lb'):
        return 4
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '7-8lb'):
        return 7
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '9lb'):
        return 9 

UPS_Rates['WEIGHT_LOWER'] = UPS_Rates.apply(g, axis=1)
UPS_Rates['WEIGHT_UPPER'] = UPS_Rates.apply(f, axis=1)


# add client margin mins column
def h(col):   
    if col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return col['MIN_WITH_REBATE']
    
    elif (col['SERVICE_LEVEL'] == 'Next Day Air') | (col['SERVICE_LEVEL'] == 'Next Day Air Saver') | (col['SERVICE_LEVEL'] == '2nd A.M. Day Air') | (col['SERVICE_LEVEL'] == '2nd Day Air') | (col['SERVICE_LEVEL'] == '3 Day Select'): 
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_AIR_BR))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'Ground Commercial') | (col['SERVICE_LEVEL'] == 'Ground Residential'): 
        try: 
            return min(10.1, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb'): 
        try: 
            return min(8.14, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR))
        except ValueError:
            return 0
    
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb'): 
        try: 
            return min(8.14, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR))
        except ValueError:
            return 0
  
    elif col['SERVICE_LEVEL'] == 'Ground CWT': 
        try: 
            return min(80, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR))
        except ValueError:
            return 0
  
    elif (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR))
        except ValueError:
            return 0

UPS_Rates['CLIENT_MARGIN_MINS'] = UPS_Rates.apply(h, axis=1)

# add ADD_MARGIN_(COL_M/DECIMAL) column
UPS_Rates['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = 1 - UPS_Rates['NET_CONTRACT_DISC']
UPS_Rates.loc[UPS_Rates['SERVICE_LEVEL'] == 'Next Day Air Early AM', 'DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = None

def i(col):   
    if (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') | (col['WEIGHT_BREAK'] == '9lb'):
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_AIR_BR)
    elif col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return
    elif (col['SERVICE_LEVEL'] == 'Ground Commercial') | (col['SERVICE_LEVEL'] == 'Ground Residential') | (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') | (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb'): 
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_GRND_BR)
    else:
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_AIR_BR)

UPS_Rates['ADD_MARGIN_(COL_M/DECIMAL)'] = UPS_Rates.apply(i, axis=1) 

# add DISCOUNT_TO_CLIENT(1-COLN) column 
def j(col):   
    if (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        return max(0, 1 - col['ADD_MARGIN_(COL_M/DECIMAL)'])
    elif col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return
    else:
        return 1 - col['ADD_MARGIN_(COL_M/DECIMAL)']

UPS_Rates['DISCOUNT_TO_CLIENT(1-COLN)'] = UPS_Rates.apply(j, axis=1)


# This completes the left side of "Rates" tab from "Customer Rate Contract Cards 2023" doc
# UPS_Rates



In [1069]:
#GRND ['WEIGHT_BREAK'] == '1-5lb'
# slice the published rates
tbl_barrettRates_1 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]

# get DISCOUNT_TO_CLIENT(1-COLN) and CLIENT_MARGIN_MINS
DISCOUNT_TO_CLIENT_1 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_1 = round(DISCOUNT_TO_CLIENT_1,2)
else:
    DISCOUNT_TO_CLIENT_1 = DISCOUNT_TO_CLIENT_1
CLIENT_MARGIN_MINS_1 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]

# use max(published rates * (1-round(DISCOUNT_TO_CLIENT)), CLIENT_MARGIN_MINS) to get the client rate
tbl_barrettRates_1['BARRETT_RATES_1'] = tbl_barrettRates_1['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
tbl_barrettRates_1['BARRETT_RATES'] = round(tbl_barrettRates_1['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
tbl_barrettRates_1 = tbl_barrettRates_1.drop('BARRETT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '6-10lb'
tbl_barrettRates_2 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_2 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_2 = round(DISCOUNT_TO_CLIENT_2,2)
else:
    DISCOUNT_TO_CLIENT_2 = DISCOUNT_TO_CLIENT_2
CLIENT_MARGIN_MINS_2 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_2['BARRETT_RATES_1'] = tbl_barrettRates_2['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_2)
tbl_barrettRates_2['BARRETT_RATES'] = round(tbl_barrettRates_2['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_2)),2)
tbl_barrettRates_2 = tbl_barrettRates_2.drop('BARRETT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '11-20lb'
tbl_barrettRates_3 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_3 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_3 = round(DISCOUNT_TO_CLIENT_3,2)
else:
    DISCOUNT_TO_CLIENT_3 = DISCOUNT_TO_CLIENT_3
CLIENT_MARGIN_MINS_3 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_3['BARRETT_RATES_1'] = tbl_barrettRates_3['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_3)
tbl_barrettRates_3['BARRETT_RATES'] = round(tbl_barrettRates_3['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_3)),2)
tbl_barrettRates_3 = tbl_barrettRates_3.drop('BARRETT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '21-30lb'
tbl_barrettRates_4 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_4 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_4 = round(DISCOUNT_TO_CLIENT_4,2)
else:
    DISCOUNT_TO_CLIENT_4 = DISCOUNT_TO_CLIENT_4
CLIENT_MARGIN_MINS_4 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_4['BARRETT_RATES_1'] = tbl_barrettRates_4['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_4)
tbl_barrettRates_4['BARRETT_RATES'] = round(tbl_barrettRates_4['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_4)),2)
tbl_barrettRates_4 = tbl_barrettRates_4.drop('BARRETT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '31+lb'
tbl_barrettRates_5 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_5 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_5 = round(DISCOUNT_TO_CLIENT_5,2)
else:
    DISCOUNT_TO_CLIENT_5 = DISCOUNT_TO_CLIENT_5
CLIENT_MARGIN_MINS_5 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_5['BARRETT_RATES_1'] = tbl_barrettRates_5['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_5)
tbl_barrettRates_5['BARRETT_RATES'] = round(tbl_barrettRates_5['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_5)),2)
tbl_barrettRates_5 = tbl_barrettRates_5.drop('BARRETT_RATES_1', axis=1)


#GRES 
#GRES ['WEIGHT_BREAK'] == '1-5lb'
tbl_barrettRates_6 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]
DISCOUNT_TO_CLIENT_6 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_6 = round(DISCOUNT_TO_CLIENT_6,2)
else:
    DISCOUNT_TO_CLIENT_6 = DISCOUNT_TO_CLIENT_6
CLIENT_MARGIN_MINS_6 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_6['BARRETT_RATES_1'] = tbl_barrettRates_6['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_6)
tbl_barrettRates_6['BARRETT_RATES'] = round(tbl_barrettRates_6['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_6)),2)
tbl_barrettRates_6 = tbl_barrettRates_6.drop('BARRETT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '6-10lb'
tbl_barrettRates_7 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_7 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_7 = round(DISCOUNT_TO_CLIENT_7,2)
else:
    DISCOUNT_TO_CLIENT_7 = DISCOUNT_TO_CLIENT_7
CLIENT_MARGIN_MINS_7 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_7['BARRETT_RATES_1'] = tbl_barrettRates_7['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_7)
tbl_barrettRates_7['BARRETT_RATES'] = round(tbl_barrettRates_7['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_7)),2)
tbl_barrettRates_7 = tbl_barrettRates_7.drop('BARRETT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '11-20lb'
tbl_barrettRates_8 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_8 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_8 = round(DISCOUNT_TO_CLIENT_8,2)
else:
    DISCOUNT_TO_CLIENT_8 = DISCOUNT_TO_CLIENT_8
CLIENT_MARGIN_MINS_8 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_8['BARRETT_RATES_1'] = tbl_barrettRates_8['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_8)
tbl_barrettRates_8['BARRETT_RATES'] = round(tbl_barrettRates_8['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_8)),2)
tbl_barrettRates_8 = tbl_barrettRates_8.drop('BARRETT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '21-30lb'
tbl_barrettRates_9 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_9 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_9 = round(DISCOUNT_TO_CLIENT_9,2)
else:
    DISCOUNT_TO_CLIENT_9 = DISCOUNT_TO_CLIENT_9
CLIENT_MARGIN_MINS_9 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_9['BARRETT_RATES_1'] = tbl_barrettRates_9['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_9)
tbl_barrettRates_9['BARRETT_RATES'] = round(tbl_barrettRates_9['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_9)),2)
tbl_barrettRates_9 = tbl_barrettRates_9.drop('BARRETT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '31+lb'
tbl_barrettRates_10 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_10 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_10 = round(DISCOUNT_TO_CLIENT_10,2)
else:
    DISCOUNT_TO_CLIENT_10 = DISCOUNT_TO_CLIENT_10
CLIENT_MARGIN_MINS_10 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_10['BARRETT_RATES_1'] = tbl_barrettRates_10['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_10)
tbl_barrettRates_10['BARRETT_RATES'] = round(tbl_barrettRates_10['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_10)),2)
tbl_barrettRates_10 = tbl_barrettRates_10.drop('BARRETT_RATES_1', axis=1)


#SRPT
#SRPT ['WEIGHT_BREAK'] == '1-3lb'
tbl_barrettRates_11 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] <= 3)]
DISCOUNT_TO_CLIENT_11 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '1-3lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_11 = round(DISCOUNT_TO_CLIENT_11,2)
else:
    DISCOUNT_TO_CLIENT_11 = DISCOUNT_TO_CLIENT_11
CLIENT_MARGIN_MINS_11 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '1-3lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_11['BARRETT_RATES_1'] = tbl_barrettRates_11['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_11)
tbl_barrettRates_11['BARRETT_RATES'] = round(tbl_barrettRates_11['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_11)),2)
tbl_barrettRates_11 = tbl_barrettRates_11.drop('BARRETT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '4-6lb'
tbl_barrettRates_12 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] >= 4) & (tbl_publishedRates['WEIGHT_LB'] <= 6)]
DISCOUNT_TO_CLIENT_12 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '4-6lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_12 = round(DISCOUNT_TO_CLIENT_12,2)
else:
    DISCOUNT_TO_CLIENT_12 = DISCOUNT_TO_CLIENT_12
CLIENT_MARGIN_MINS_12 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '4-6lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_12['BARRETT_RATES_1'] = tbl_barrettRates_12['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_12)
tbl_barrettRates_12['BARRETT_RATES'] = round(tbl_barrettRates_12['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_12)),2)
tbl_barrettRates_12 = tbl_barrettRates_12.drop('BARRETT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '7-8lb'
tbl_barrettRates_13 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] >= 7) & (tbl_publishedRates['WEIGHT_LB'] <= 8)]
DISCOUNT_TO_CLIENT_13 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '7-8lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_13 = round(DISCOUNT_TO_CLIENT_13,2)
else:
    DISCOUNT_TO_CLIENT_13 = DISCOUNT_TO_CLIENT_13
CLIENT_MARGIN_MINS_13 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '7-8lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_13['BARRETT_RATES_1'] = tbl_barrettRates_13['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_13)
tbl_barrettRates_13['BARRETT_RATES'] = round(tbl_barrettRates_13['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_13)),2)
tbl_barrettRates_13 = tbl_barrettRates_13.drop('BARRETT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '9lb'
tbl_barrettRates_14 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] == 9)]
DISCOUNT_TO_CLIENT_14 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '9lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_14 = round(DISCOUNT_TO_CLIENT_14,2)
else:
    DISCOUNT_TO_CLIENT_14 = DISCOUNT_TO_CLIENT_14
CLIENT_MARGIN_MINS_14 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '9lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_14['BARRETT_RATES_1'] = tbl_barrettRates_14['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_14)
tbl_barrettRates_14['BARRETT_RATES'] = round(tbl_barrettRates_14['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_14)),2)
tbl_barrettRates_14 = tbl_barrettRates_14.drop('BARRETT_RATES_1', axis=1)


#Others:
#UPS REDE, there is no discount, therefore REDE's client rate = published rate
tbl_barrettRates_15 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'REDE')]
tbl_barrettRates_15['BARRETT_RATES'] = tbl_barrettRates_15['PUBLISHED_RATES']


#UPS RED
tbl_barrettRates_16 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'RED')]
DISCOUNT_TO_CLIENT_16 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'RED'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_16 = round(DISCOUNT_TO_CLIENT_16,2)
else:
    DISCOUNT_TO_CLIENT_16 = DISCOUNT_TO_CLIENT_16
CLIENT_MARGIN_MINS_16 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'RED'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_16['BARRETT_RATES_1'] = tbl_barrettRates_16['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_16)
tbl_barrettRates_16['BARRETT_RATES'] = round(tbl_barrettRates_16['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_16)),2)
tbl_barrettRates_16 = tbl_barrettRates_16.drop('BARRETT_RATES_1', axis=1)


#UPS REDS, this does not have round up option (1/2)
tbl_barrettRates_17 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'REDS')]
DISCOUNT_TO_CLIENT_17 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'REDS'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_17 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'REDS'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_17['BARRETT_RATES_1'] = tbl_barrettRates_17['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_17)
tbl_barrettRates_17['BARRETT_RATES'] = round(tbl_barrettRates_17['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_17)),2)
tbl_barrettRates_17 = tbl_barrettRates_17.drop('BARRETT_RATES_1', axis=1)


#UPS 2DAM, this does not have round up option (2/2)
tbl_barrettRates_18 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '2DAM')]
DISCOUNT_TO_CLIENT_18 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == '2DAM'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_18 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == '2DAM'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_18['BARRETT_RATES_1'] = tbl_barrettRates_18['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_18)
tbl_barrettRates_18['BARRETT_RATES'] = round(tbl_barrettRates_18['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_18)),2)
tbl_barrettRates_18 = tbl_barrettRates_18.drop('BARRETT_RATES_1', axis=1)


#UPS BLUE
tbl_barrettRates_19 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'BLUE')]
DISCOUNT_TO_CLIENT_19 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'BLUE'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_19 = round(DISCOUNT_TO_CLIENT_19,2)
else:
    DISCOUNT_TO_CLIENT_19 = DISCOUNT_TO_CLIENT_19
CLIENT_MARGIN_MINS_19 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'BLUE') , 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_19['BARRETT_RATES_1'] = tbl_barrettRates_19['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_19)
tbl_barrettRates_19['BARRETT_RATES'] = round(tbl_barrettRates_19['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_19)),2)
tbl_barrettRates_19 = tbl_barrettRates_19.drop('BARRETT_RATES_1', axis=1)


#UPS ORNG
tbl_barrettRates_20 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'ORNG')]
DISCOUNT_TO_CLIENT_20 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'ORNG'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_20 = round(DISCOUNT_TO_CLIENT_20,2)
else:
    DISCOUNT_TO_CLIENT_20 = DISCOUNT_TO_CLIENT_20
CLIENT_MARGIN_MINS_20 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'ORNG'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_20['BARRETT_RATES_1'] = tbl_barrettRates_20['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_20)
tbl_barrettRates_20['BARRETT_RATES'] = round(tbl_barrettRates_20['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_20)),2)
tbl_barrettRates_20 = tbl_barrettRates_20.drop('BARRETT_RATES_1', axis=1)


#UPS SRPT<1
tbl_barrettRates_21 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT<1')]
DISCOUNT_TO_CLIENT_21 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT<1'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_21 = round(DISCOUNT_TO_CLIENT_21,2)
else:
    DISCOUNT_TO_CLIENT_21 = DISCOUNT_TO_CLIENT_21
CLIENT_MARGIN_MINS_21 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT<1'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_21['BARRETT_RATES_1'] = tbl_barrettRates_21['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_21)
tbl_barrettRates_21['BARRETT_RATES'] = round(tbl_barrettRates_21['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_21)),2)
tbl_barrettRates_21 = tbl_barrettRates_21.drop('BARRETT_RATES_1', axis=1)

C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\1387918324.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_barrettRates_1['BARRETT_RATES_1'] = tbl_barrettRates_1['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\1387918324.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_barrettRates_1['BARRETT_RATES'] = round(tbl_barrettRates_1['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\138

In [1070]:
#Get dhl, usps, and mi rates
tbl_barrettRates_22 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'USPSAG') | 
                                         (tbl_publishedRates['SERVICE_CODE'] == 'USPSAP') | 
                                         (tbl_publishedRates['SERVICE_CODE'] == 'USPSAG_CI') | 
                                         (tbl_publishedRates['SERVICE_CODE'] == 'USPSAP_CI') | 
                                         (tbl_publishedRates['SERVICE_CODE'] == 'MIPLE') |
                                         (tbl_publishedRates['SERVICE_CODE'] == 'MIPH') |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLG_'))   |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLG<1_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLE_'))   |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLE<1_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLEM_'))]


#combine them to rename as BarrettRates
tbl_BarrettRates = pd.concat([tbl_barrettRates_1, tbl_barrettRates_2, tbl_barrettRates_3, tbl_barrettRates_4, 
                    tbl_barrettRates_5, tbl_barrettRates_6, tbl_barrettRates_7, tbl_barrettRates_8,
                    tbl_barrettRates_9, tbl_barrettRates_10, tbl_barrettRates_11, tbl_barrettRates_12,
                    tbl_barrettRates_13, tbl_barrettRates_14, tbl_barrettRates_15, tbl_barrettRates_16,
                    tbl_barrettRates_17, tbl_barrettRates_18, tbl_barrettRates_19, tbl_barrettRates_20,
                    tbl_barrettRates_21, tbl_barrettRates_22])
#Keep this table as it only contains published rates and Barrett rates
#tbl_BarrettRates.head()

In [1071]:

# Define two different mappings
mapping_for_ground = {0.1: 'Tier1', 0.2: 'Tier2', 0.3: 'Tier3', 0.4: 'Tier4', 0.5: 'Tier5', 0.6: 'Tier6', 0.7: 'Tier7', 
                      0.8: 'Tier8', 0.9: 'Tier9', 1: 'Tier10'}

mapping_for_priority = {0.1: 'Tier1', 0.2: 'Tier2', 0.3: 'Tier3', 0.4: 'Tier4', 0.5: 'Tier5'}

def apply_custom_mapping(row):
    if row['SERVICE_CODE'] == 'USPSAG_CI':
        return mapping_for_ground.get(row['CUBIC_FT'], None)  # None for default
    elif row['SERVICE_CODE'] == 'USPSAP_CI':
        return mapping_for_priority.get(row['CUBIC_FT'], None)  # None for default
    else:
        return None  # Or some default value for other ServiceCodes

# Apply the custom function to create a new column
tbl_publishedRates['TIER'] = tbl_publishedRates.apply(apply_custom_mapping, axis=1)
tbl_BarrettRates['TIER'] = tbl_BarrettRates.apply(apply_custom_mapping, axis=1)



In [1072]:
#Temp solution to handle rounding difference
tbl_BarrettRates.to_csv('tbl_BarrettRates.csv')
tbl_publishedRates.to_csv('tbl_publishedRates.csv')
tbl_BarrettRates = pd.read_csv('C:\\Users\\sliu\\BD\\re-rate_model\\tbl_BarrettRates.csv')
tbl_publishedRates = pd.read_csv('C:\\Users\\sliu\\BD\\re-rate_model\\tbl_publishedRates.csv')
tbl_BarrettRates['ZONE'] = tbl_BarrettRates['ZONE'].astype(str)
tbl_BarrettRates['VERSION'] = tbl_BarrettRates['VERSION'].astype(str)
tbl_publishedRates['ZONE'] = tbl_publishedRates['ZONE'].astype(str)
tbl_publishedRates['VERSION'] = tbl_publishedRates['VERSION'].astype(str)

# Step 1 - c. Create Freight Rate Card (tbl_clientRates)
*Add ups, dhl, and usps clientRates in tbl_publishedRates. Read the margins located at the top of this model. 

In [1073]:
# UPSAccessorial_2023 has 2 tabs, UPS_Rates and UPS_Accessorial. UPS_Rates is used to create Barrett Rates
xls = pd.ExcelFile('UPSAccessorial_2023.xlsx')
UPS_Rates = pd.read_excel(xls, 'UPS_Rates')

In [1074]:
#add 2 columns to this table with min and max weight
def f(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 5
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 10
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 20
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 30
    elif (col['WEIGHT_BREAK'] == '31+lb'):
        return 999
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-16oz'):
        return 16
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-3lb'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-6lb'):
        return 6
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '7-8lb'):
        return 8
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '9lb'):
        return 9  

def g(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 1
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 6
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 11
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 21
    elif col['WEIGHT_BREAK'] == '31+lb':
        return 31
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-16oz'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-3lb'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-6lb'):
        return 4
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '7-8lb'):
        return 7
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '9lb'):
        return 9 

UPS_Rates['WEIGHT_LOWER'] = UPS_Rates.apply(g, axis=1)
UPS_Rates['WEIGHT_UPPER'] = UPS_Rates.apply(f, axis=1)


# add client margin mins column
def h(col):   
    if col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return col['MIN_WITH_REBATE']
    
    elif (col['SERVICE_LEVEL'] == 'Next Day Air') | (col['SERVICE_LEVEL'] == 'Next Day Air Saver') | (col['SERVICE_LEVEL'] == '2nd A.M. Day Air') | (col['SERVICE_LEVEL'] == '2nd Day Air') | (col['SERVICE_LEVEL'] == '3 Day Select'): 
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_AIR))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'Ground Commercial') | (col['SERVICE_LEVEL'] == 'Ground Residential'): 
        try: 
            return min(10.1, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb'): 
        try: 
            return min(8.14, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND))
        except ValueError:
            return 0
        
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb'): 
        try: 
            return min(8.14, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND))
        except ValueError:
            return 0
  
    elif col['SERVICE_LEVEL'] == 'Ground CWT': 
        try: 
            return min(80, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND))
        except ValueError:
            return 0
  
    elif (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND))
        except ValueError:
            return 0

UPS_Rates['CLIENT_MARGIN_MINS'] = UPS_Rates.apply(h, axis=1)

# add ADD_MARGIN_(COL_M/DECIMAL) column
UPS_Rates['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = 1 - UPS_Rates['NET_CONTRACT_DISC']
UPS_Rates.loc[UPS_Rates['SERVICE_LEVEL'] == 'Next Day Air Early AM', 'DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = None

def i(col):   
    if (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') | (col['WEIGHT_BREAK'] == '9lb'):
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_AIR)
    elif col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return
    elif (col['SERVICE_LEVEL'] == 'Ground Commercial') | (col['SERVICE_LEVEL'] == 'Ground Residential') | (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') | (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb'): 
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_GRND)
    else:
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_AIR)

UPS_Rates['ADD_MARGIN_(COL_M/DECIMAL)'] = UPS_Rates.apply(i, axis=1) 

# add DISCOUNT_TO_CLIENT(1-COLN) column 
def j(col):   
    if (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        return max(0, 1 - col['ADD_MARGIN_(COL_M/DECIMAL)'])
    elif col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return
    else:
        return 1 - col['ADD_MARGIN_(COL_M/DECIMAL)']

UPS_Rates['DISCOUNT_TO_CLIENT(1-COLN)'] = UPS_Rates.apply(j, axis=1)


# This is the left side of "Rates" tab from "Customer Rate Contract Cards 2023" doc
# UPS_Rates

In [1075]:
#GRND ['WEIGHT_BREAK'] == '1-5lb'
# slice the published rates
tbl_clientRates_1 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]

# get DISCOUNT_TO_CLIENT(1-COLN) and CLIENT_MARGIN_MINS
DISCOUNT_TO_CLIENT_1 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_1 = round(DISCOUNT_TO_CLIENT_1,2)
else:
    DISCOUNT_TO_CLIENT_1 = DISCOUNT_TO_CLIENT_1
CLIENT_MARGIN_MINS_1 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]

# use max(published rates * (1-round(DISCOUNT_TO_CLIENT)), CLIENT_MARGIN_MINS) to get the client rate
tbl_clientRates_1['CLIENT_RATES_1'] = tbl_clientRates_1['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
tbl_clientRates_1['CLIENT_RATES'] = round(tbl_clientRates_1['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
tbl_clientRates_1 = tbl_clientRates_1.drop('CLIENT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '6-10lb'
tbl_clientRates_2 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_2 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_2 = round(DISCOUNT_TO_CLIENT_2,2)
else:
    DISCOUNT_TO_CLIENT_2 = DISCOUNT_TO_CLIENT_2
CLIENT_MARGIN_MINS_2 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_2['CLIENT_RATES_1'] = tbl_clientRates_2['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_2)
tbl_clientRates_2['CLIENT_RATES'] = round(tbl_clientRates_2['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_2)),2)
tbl_clientRates_2 = tbl_clientRates_2.drop('CLIENT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '11-20lb'
tbl_clientRates_3 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_3 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_3 = round(DISCOUNT_TO_CLIENT_3,2)
else:
    DISCOUNT_TO_CLIENT_3 = DISCOUNT_TO_CLIENT_3
CLIENT_MARGIN_MINS_3 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_3['CLIENT_RATES_1'] = tbl_clientRates_3['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_3)
tbl_clientRates_3['CLIENT_RATES'] = round(tbl_clientRates_3['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_3)),2)
tbl_clientRates_3 = tbl_clientRates_3.drop('CLIENT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '21-30lb'
tbl_clientRates_4 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_4 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_4 = round(DISCOUNT_TO_CLIENT_4,2)
else:
    DISCOUNT_TO_CLIENT_4 = DISCOUNT_TO_CLIENT_4
CLIENT_MARGIN_MINS_4 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_4['CLIENT_RATES_1'] = tbl_clientRates_4['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_4)
tbl_clientRates_4['CLIENT_RATES'] = round(tbl_clientRates_4['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_4)),2)
tbl_clientRates_4 = tbl_clientRates_4.drop('CLIENT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '31+lb'
tbl_clientRates_5 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_5 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_5 = round(DISCOUNT_TO_CLIENT_5,2)
else:
    DISCOUNT_TO_CLIENT_5 = DISCOUNT_TO_CLIENT_5
CLIENT_MARGIN_MINS_5 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_5['CLIENT_RATES_1'] = tbl_clientRates_5['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_5)
tbl_clientRates_5['CLIENT_RATES'] = round(tbl_clientRates_5['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_5)),2)
tbl_clientRates_5 = tbl_clientRates_5.drop('CLIENT_RATES_1', axis=1)


#GRES 
#GRES ['WEIGHT_BREAK'] == '1-5lb'
tbl_clientRates_6 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]
DISCOUNT_TO_CLIENT_6 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_6 = round(DISCOUNT_TO_CLIENT_6,2)
else:
    DISCOUNT_TO_CLIENT_6 = DISCOUNT_TO_CLIENT_6
CLIENT_MARGIN_MINS_6 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_6['CLIENT_RATES_1'] = tbl_clientRates_6['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_6)
tbl_clientRates_6['CLIENT_RATES'] = round(tbl_clientRates_6['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_6)),2)
tbl_clientRates_6 = tbl_clientRates_6.drop('CLIENT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '6-10lb'
tbl_clientRates_7 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_7 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_7 = round(DISCOUNT_TO_CLIENT_7,2)
else:
    DISCOUNT_TO_CLIENT_7 = DISCOUNT_TO_CLIENT_7
CLIENT_MARGIN_MINS_7 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_7['CLIENT_RATES_1'] = tbl_clientRates_7['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_7)
tbl_clientRates_7['CLIENT_RATES'] = round(tbl_clientRates_7['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_7)),2)
tbl_clientRates_7 = tbl_clientRates_7.drop('CLIENT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '11-20lb'
tbl_clientRates_8 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_8 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_8 = round(DISCOUNT_TO_CLIENT_8,2)
else:
    DISCOUNT_TO_CLIENT_8 = DISCOUNT_TO_CLIENT_8
CLIENT_MARGIN_MINS_8 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_8['CLIENT_RATES_1'] = tbl_clientRates_8['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_8)
tbl_clientRates_8['CLIENT_RATES'] = round(tbl_clientRates_8['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_8)),2)
tbl_clientRates_8 = tbl_clientRates_8.drop('CLIENT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '21-30lb'
tbl_clientRates_9 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_9 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_9 = round(DISCOUNT_TO_CLIENT_9,2)
else:
    DISCOUNT_TO_CLIENT_9 = DISCOUNT_TO_CLIENT_9
CLIENT_MARGIN_MINS_9 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_9['CLIENT_RATES_1'] = tbl_clientRates_9['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_9)
tbl_clientRates_9['CLIENT_RATES'] = round(tbl_clientRates_9['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_9)),2)
tbl_clientRates_9 = tbl_clientRates_9.drop('CLIENT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '31+lb'
tbl_clientRates_10 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_10 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_10 = round(DISCOUNT_TO_CLIENT_10,2)
else:
    DISCOUNT_TO_CLIENT_10 = DISCOUNT_TO_CLIENT_10
CLIENT_MARGIN_MINS_10 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_10['CLIENT_RATES_1'] = tbl_clientRates_10['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_10)
tbl_clientRates_10['CLIENT_RATES'] = round(tbl_clientRates_10['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_10)),2)
tbl_clientRates_10 = tbl_clientRates_10.drop('CLIENT_RATES_1', axis=1)


#SRPT
#SRPT ['WEIGHT_BREAK'] == '1-3lb'
tbl_clientRates_11 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] <= 3)]
DISCOUNT_TO_CLIENT_11 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '1-3lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_11 = round(DISCOUNT_TO_CLIENT_11,2)
else:
    DISCOUNT_TO_CLIENT_11 = DISCOUNT_TO_CLIENT_11
CLIENT_MARGIN_MINS_11 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '1-3lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_11['CLIENT_RATES_1'] = tbl_clientRates_11['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_11)
tbl_clientRates_11['CLIENT_RATES'] = round(tbl_clientRates_11['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_11)),2)
tbl_clientRates_11 = tbl_clientRates_11.drop('CLIENT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '4-6lb'
tbl_clientRates_12 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] >= 4) & (tbl_publishedRates['WEIGHT_LB'] <= 6)]
DISCOUNT_TO_CLIENT_12 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '4-6lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_12 = round(DISCOUNT_TO_CLIENT_12,2)
else:
    DISCOUNT_TO_CLIENT_12 = DISCOUNT_TO_CLIENT_12
CLIENT_MARGIN_MINS_12 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '4-6lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_12['CLIENT_RATES_1'] = tbl_clientRates_12['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_12)
tbl_clientRates_12['CLIENT_RATES'] = round(tbl_clientRates_12['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_12)),2)
tbl_clientRates_12 = tbl_clientRates_12.drop('CLIENT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '7-8lb'
tbl_clientRates_13 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] >= 7) & (tbl_publishedRates['WEIGHT_LB'] <= 8)]
DISCOUNT_TO_CLIENT_13 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '7-8lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_13 = round(DISCOUNT_TO_CLIENT_13,2)
else:
    DISCOUNT_TO_CLIENT_13 = DISCOUNT_TO_CLIENT_13
CLIENT_MARGIN_MINS_13 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '7-8lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_13['CLIENT_RATES_1'] = tbl_clientRates_13['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_13)
tbl_clientRates_13['CLIENT_RATES'] = round(tbl_clientRates_13['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_13)),2)
tbl_clientRates_13 = tbl_clientRates_13.drop('CLIENT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '9lb'
tbl_clientRates_14 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] == 9)]
DISCOUNT_TO_CLIENT_14 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '9lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_14 = round(DISCOUNT_TO_CLIENT_14,2)
else:
    DISCOUNT_TO_CLIENT_14 = DISCOUNT_TO_CLIENT_14
CLIENT_MARGIN_MINS_14 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '9lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_14['CLIENT_RATES_1'] = tbl_clientRates_14['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_14)
tbl_clientRates_14['CLIENT_RATES'] = round(tbl_clientRates_14['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_14)),2)
tbl_clientRates_14 = tbl_clientRates_14.drop('CLIENT_RATES_1', axis=1)


#Others:
#UPS REDE, there is no discount, therefore REDE's client rate = published rate
tbl_clientRates_15 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'REDE')]
tbl_clientRates_15['CLIENT_RATES'] = tbl_clientRates_15['PUBLISHED_RATES']


#UPS RED
tbl_clientRates_16 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'RED')]
DISCOUNT_TO_CLIENT_16 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'RED'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_16 = round(DISCOUNT_TO_CLIENT_16,2)
else:
    DISCOUNT_TO_CLIENT_16 = DISCOUNT_TO_CLIENT_16
CLIENT_MARGIN_MINS_16 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'RED'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_16['CLIENT_RATES_1'] = tbl_clientRates_16['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_16)
tbl_clientRates_16['CLIENT_RATES'] = round(tbl_clientRates_16['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_16)),2)
tbl_clientRates_16 = tbl_clientRates_16.drop('CLIENT_RATES_1', axis=1)


#UPS REDS, this does not have round up option (1/2)
tbl_clientRates_17 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'REDS')]
DISCOUNT_TO_CLIENT_17 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'REDS'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_17 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'REDS'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_17['CLIENT_RATES_1'] = tbl_clientRates_17['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_17)
tbl_clientRates_17['CLIENT_RATES'] = round(tbl_clientRates_17['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_17)),2)
tbl_clientRates_17 = tbl_clientRates_17.drop('CLIENT_RATES_1', axis=1)


#UPS 2DAM, this does not have round up option (2/2)
tbl_clientRates_18 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '2DAM')]
DISCOUNT_TO_CLIENT_18 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == '2DAM'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_18 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == '2DAM'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_18['CLIENT_RATES_1'] = tbl_clientRates_18['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_18)
tbl_clientRates_18['CLIENT_RATES'] = round(tbl_clientRates_18['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_18)),2)
tbl_clientRates_18 = tbl_clientRates_18.drop('CLIENT_RATES_1', axis=1)


#UPS BLUE
tbl_clientRates_19 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'BLUE')]
DISCOUNT_TO_CLIENT_19 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'BLUE'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_19 = round(DISCOUNT_TO_CLIENT_19,2)
else:
    DISCOUNT_TO_CLIENT_19 = DISCOUNT_TO_CLIENT_19
CLIENT_MARGIN_MINS_19 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'BLUE') , 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_19['CLIENT_RATES_1'] = tbl_clientRates_19['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_19)
tbl_clientRates_19['CLIENT_RATES'] = round(tbl_clientRates_19['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_19)),2)
tbl_clientRates_19 = tbl_clientRates_19.drop('CLIENT_RATES_1', axis=1)


#UPS ORNG
tbl_clientRates_20 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'ORNG')]
DISCOUNT_TO_CLIENT_20 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'ORNG'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_20 = round(DISCOUNT_TO_CLIENT_20,2)
else:
    DISCOUNT_TO_CLIENT_20 = DISCOUNT_TO_CLIENT_20
CLIENT_MARGIN_MINS_20 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'ORNG'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_20['CLIENT_RATES_1'] = tbl_clientRates_20['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_20)
tbl_clientRates_20['CLIENT_RATES'] = round(tbl_clientRates_20['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_20)),2)
tbl_clientRates_20 = tbl_clientRates_20.drop('CLIENT_RATES_1', axis=1)


#UPS SRPT<1
tbl_clientRates_21 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT<1')]
DISCOUNT_TO_CLIENT_21 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT<1'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_21 = round(DISCOUNT_TO_CLIENT_21,2)
else:
    DISCOUNT_TO_CLIENT_21 = DISCOUNT_TO_CLIENT_21
CLIENT_MARGIN_MINS_21 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT<1'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_21['CLIENT_RATES_1'] = tbl_clientRates_21['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_21)
tbl_clientRates_21['CLIENT_RATES'] = round(tbl_clientRates_21['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_21)),2)
tbl_clientRates_21 = tbl_clientRates_21.drop('CLIENT_RATES_1', axis=1)



tbl_ups_clientRates = pd.concat([tbl_clientRates_1, tbl_clientRates_2, tbl_clientRates_3, tbl_clientRates_4, 
                    tbl_clientRates_5, tbl_clientRates_6, tbl_clientRates_7, tbl_clientRates_8,
                    tbl_clientRates_9, tbl_clientRates_10, tbl_clientRates_11, tbl_clientRates_12,
                    tbl_clientRates_13, tbl_clientRates_14, tbl_clientRates_15, tbl_clientRates_16,
                    tbl_clientRates_17, tbl_clientRates_18, tbl_clientRates_19, tbl_clientRates_20,
                    tbl_clientRates_21])

C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\2791379907.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_clientRates_1['CLIENT_RATES_1'] = tbl_clientRates_1['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\2791379907.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_clientRates_1['CLIENT_RATES'] = round(tbl_clientRates_1['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\2791379907

C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\2791379907.py:222: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_clientRates_18['CLIENT_RATES_1'] = tbl_clientRates_18['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_18)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\2791379907.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_clientRates_18['CLIENT_RATES'] = round(tbl_clientRates_18['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_18)),2)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\27

In [1076]:
# Create DHL and USPS Client Rate
tbl_dhl_clientRates = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLG_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLG<1_')) | 
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLE_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLE<1_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLEM_'))]

tbl_usps_clientRates = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'USPSAG') | 
                                          (tbl_publishedRates['SERVICE_CODE'] == 'USPSAP') |
                                          (tbl_publishedRates['SERVICE_CODE'] == 'USPSAG_CI') | 
                                          (tbl_publishedRates['SERVICE_CODE'] == 'USPSAP_CI')]


tbl_mi_clientRates = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'MIPLE') | 
                                          (tbl_publishedRates['SERVICE_CODE'] == 'MIPH')]


tbl_dhl_clientRates['CLIENT_RATES'] = round(tbl_dhl_clientRates['BARRETT_RATES']/(1 - Margin_on_DHL),2)
tbl_usps_clientRates['CLIENT_RATES'] = round(tbl_usps_clientRates['BARRETT_RATES']/(1 - Margin_on_USPS),2)
tbl_mi_clientRates['CLIENT_RATES'] = round(tbl_mi_clientRates['BARRETT_RATES']/(1 - Margin_on_MI),2)


tbl_clientRates = pd.concat([tbl_ups_clientRates, tbl_dhl_clientRates, tbl_usps_clientRates, tbl_mi_clientRates])
tbl_clientRates = tbl_clientRates.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'CUBIC_FT', 'PUBLISHED_RATES', 'CLIENT_RATES', 'VERSION', 'START_DATE', 'END_DATE'])

#get the finalized tbl_publishedRates
tbl_clientRates = tbl_BarrettRates.merge(tbl_clientRates[['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'CUBIC_FT', 'VERSION', 'START_DATE', 'END_DATE', 'CLIENT_RATES']], left_on=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'VERSION', 'START_DATE', 'END_DATE', 'CUBIC_FT'], right_on=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'VERSION', 'START_DATE', 'END_DATE', 'CUBIC_FT'], how='left')
#tbl_clientRates.to_csv('test.csv')



C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\6757182.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_dhl_clientRates['CLIENT_RATES'] = round(tbl_dhl_clientRates['BARRETT_RATES']/(1 - Margin_on_DHL),2)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\6757182.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_usps_clientRates['CLIENT_RATES'] = round(tbl_usps_clientRates['BARRETT_RATES']/(1 - Margin_on_USPS),2)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\6757182.py:20: SettingWithCopyWa

# Step 2 - a. Create UPS accessorial rates
*This rate is separated from freight rate

In [1077]:
# UPSAccessorial_2023 has 2 tabs, UPS_Rates and UPS_Accessorial
xls = pd.ExcelFile('UPSAccessorial_2023.xlsx')
UPS_Accessorial = pd.read_excel(xls, 'UPS_Accessorial')

# Add Net_charge column
UPS_Accessorial['NET_CHARGE'] = UPS_Accessorial['PUBLISHED_CHARGE']*(1-UPS_Accessorial['DISCOUNT'])

# Add ADD_MARGIN column
def k(col):   
    if UPS_Accessorial['ACCESSORIAL'].str.contains('Air').any():
        return col['NET_CHARGE']/(1-Margin_on_Value_Add_AIR)
    else:
        return col['NET_CHARGE']/(1-Margin_on_Value_Add_GRND)

UPS_Accessorial['ADD_MARGIN'] = UPS_Accessorial.apply(k, axis=1)  


# Assign a number to a specific cell based on column name and other row values
column_name = 'ADD_MARGIN'
condition1 = UPS_Accessorial['ACCESSORIAL'] == 'Remote Area' 
condition2 = UPS_Accessorial['ACCESSORIAL'] == 'Remote Area Alaska' 
condition3 = UPS_Accessorial['ACCESSORIAL'] == 'Remote Area Hawaii' 

UPS_Accessorial.loc[condition1, column_name] = 13.05
UPS_Accessorial.loc[condition2, column_name] = 38.00
UPS_Accessorial.loc[condition3, column_name] = 13.05


# Add PERCENTAGE column
def l(col):   
    if Round_Discount == 'YES':
        return 1 - round((col['ADD_MARGIN']/col['PUBLISHED_CHARGE']),2)
    else:
        return 1 - col['ADD_MARGIN']/col['PUBLISHED_CHARGE']

UPS_Accessorial['PERCENTAGE'] = UPS_Accessorial.apply(l, axis=1)

# Step 2 - b. Prepare all other rates for rerate
*These rates do not need a table to house

In [1078]:
# List all UPS Surcharge rates for rerate
Resi_Surcharge_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Resi Surcharge (Air)'), 'ADD_MARGIN'].values[0]
Resi_Surcharge_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Resi Surcharge (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Comm_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Comm (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Comm_Extended_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Comm Extended (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Resi (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Extended_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Resi Extended (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Comm_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Comm (Air)'), 'ADD_MARGIN'].values[0]
DAS_Comm_Extended_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Comm Extended (Air)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Resi (Air)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Extended_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Resi Extended (Air)'), 'ADD_MARGIN'].values[0]
DAS_SurePost = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS SurePost'), 'ADD_MARGIN'].values[0]
DAS_Extended_SurePost = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Extended SurePost'), 'ADD_MARGIN'].values[0]
Remote_Area = 13.05
Remote_Area_Alaska = 38.00
Remote_Area_Hawaii = 13.05


# Get FSC ready for rerate
xls = pd.ExcelFile('FSC.xlsx')
UPS_FSC = pd.read_excel(xls, 'UPS_FSC')
DHL_FSC = pd.read_excel(xls, 'DHL_FSC')

#Get UPS_FSC doc ready
UPS_FSC['SHIP_DATE_START'] = UPS_FSC['Ship Date']
UPS_FSC['Ship Date'] = pd.to_datetime(UPS_FSC['Ship Date']) 
UPS_FSC['SHIP_DATE_END'] = UPS_FSC['Ship Date'] + pd.Timedelta(days=6)
UPS_FSC = UPS_FSC.drop('Ship Date', axis=1)

UPS_FSC['SHIP_DATE_START'] = pd.to_datetime(UPS_FSC['SHIP_DATE_START']).dt.date
UPS_FSC['SHIP_DATE_END'] = pd.to_datetime(UPS_FSC['SHIP_DATE_END']).dt.date

#Get DHL_FSC doc ready
from datetime import timedelta
from datetime import date
DHL_FSC = DHL_FSC.sort_values(by='Ship Date', ascending=True)
DHL_FSC['Ship Date'] = pd.to_datetime(DHL_FSC['Ship Date']) 
DHL_FSC['SHIP_DATE_START'] = DHL_FSC['Ship Date']
DHL_FSC['SHIP_DATE_END'] = DHL_FSC['SHIP_DATE_START'].shift(-1) - timedelta(days=1)

# Calculate a date that is 2 years in the future
future_date = date.today().replace(year=date.today().year + 2)
# Update the 'SHIP_DATE_END' column for rows with the latest 'SHIP_DATE_START'
DHL_FSC.loc[DHL_FSC['SHIP_DATE_START'] == max(DHL_FSC['SHIP_DATE_START']), ['SHIP_DATE_END']] = future_date

DHL_FSC['SHIP_DATE_END'] = pd.to_datetime(DHL_FSC['SHIP_DATE_END']) 
DHL_FSC = DHL_FSC.drop('Ship Date', axis=1)

# List UPS AHS rates for rerate
AH_Girth_2 = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling Girth 2'), 'ADD_MARGIN'].values[0]
AH_Girth_3_4 = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling Girth 3-4'), 'ADD_MARGIN'].values[0]
AH_Girth_5_up = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling Girth 5+'), 'ADD_MARGIN'].values[0]

AH_L_2 = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling L 2'), 'ADD_MARGIN'].values[0]
AH_L_3_4 = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling L 3-4'), 'ADD_MARGIN'].values[0]
AH_L_5_up = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling L 5+'), 'ADD_MARGIN'].values[0]

AH_W_2 = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling W 2'), 'ADD_MARGIN'].values[0]
AH_W_3_4 = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling W 3-4'), 'ADD_MARGIN'].values[0]
AH_W_5_up = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling W 5+'), 'ADD_MARGIN'].values[0]

AH_WGT_2 = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling WGT 2'), 'ADD_MARGIN'].values[0]
AH_WGT_3_4 = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling WGT 3-4'), 'ADD_MARGIN'].values[0]
AH_WGT_5_up = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling WGT 5+'), 'ADD_MARGIN'].values[0]


# Step 3 - a. Prepare PLD for rerate
* Regardless of various types of PLD, Client PLD, Barrett PLD, etc.

# Load PLD

In [1079]:
#change the corresponding column names to below names before upload. 
#Requested columns and the corresponding type:     
# ---  ------             --------------  -----         
#1   TRACKING_NUMBER    
#2   SHIP_DATE             
#3   OLD_SERVICE --(optional)      
#4   ZIP_CODE               
#5   ZONE --(optional)               
#6   DIMENSIONS or LWH           
#7   ACTUAL_WEIGHT               
#8   RESIDENTIAL_FLAG       
#9   INTERNATIONAL_FLAG    

#Load Sample PLD
xls = pd.ExcelFile('QuotePLD_2023.xlsx')
tbl_BP_Master = pd.read_excel(xls, 'Sheet1')
#tbl_BP.head()

In [1080]:
#When the original file is too large, make a copy
tbl_BP = tbl_BP_Master.copy()

In [1081]:
# clean the empties if necessary
tbl_BP = tbl_BP.dropna(axis = 0, how = 'all')
tbl_BP = tbl_BP.dropna(axis = 1, how = 'all')

In [1082]:
tbl_BP.shape

(126511, 21)

# Clean PLD

In [1083]:
#The column type must match
#Requested columns and the corresponding type:     
# ---  ------             --------------  -----         
#1   TRACKING_NUMBER        ---------------------      int64 (or object) "can manually create one before upload"       
#2   SHIP_DATE              ---------------------      datetime64[ns] "can manually fill up before upload"
#3   OLD_SERVICE (optional) ---------------------      object        
#4   ZIP_CODE               ---------------------      object
#5   ZONE (optional)        ---------------------      object          
#6   DIMENSIONS (or LWH)    ---------------------      object (or float64 or int64)       
#7   ACTUAL_WEIGHT (lb as default)---------------      float64       
#8   RESIDENTIAL_FLAG       ---------------------      bool   
tbl_BP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126511 entries, 0 to 126510
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CustomerID          126511 non-null  int64         
 1   CustomerName        126511 non-null  object        
 2   Facility            126511 non-null  object        
 3   BarrettOrderNumber  126511 non-null  object        
 4   Reference           126511 non-null  object        
 5   PoNumber            117175 non-null  object        
 6   TRACKING_NUMBER     126511 non-null  object        
 7   SHIP_DATE           126511 non-null  datetime64[ns]
 8   OLD_SERVICE         126511 non-null  object        
 9   Shipper             126511 non-null  object        
 10  ShipToName          126511 non-null  object        
 11  Contact             27292 non-null   object        
 12  City                126511 non-null  object        
 13  State               126511 no

In [1084]:
#update column type
tbl_BP['SHIP_DATE'] = pd.to_datetime(tbl_BP['SHIP_DATE'])
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str)
tbl_BP['ACTUAL_WEIGHT'] = tbl_BP['ACTUAL_WEIGHT'].astype(float)
tbl_BP['RESIDENTIAL_FLAG'] = tbl_BP['RESIDENTIAL_FLAG'].astype('bool')

In [1085]:
#Check the Zip_Code values
tbl_BP['ZIP_CODE'].unique() 

array(['77433', '12883', '6762', ..., '47841', '8609', '50060'],
      dtype=object)

In [1086]:
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str).str.replace('\.0$', '').astype(int)
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str)

C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\2751569339.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str).str.replace('\.0$', '').astype(int)


In [1087]:
#Check the value of Zone if exists
tbl_BP['ZONE'].unique()

array(['4.0', '6.0', '7.0', '5.0', '2.0', '3.0', '8.0', 'nan', '1.0',
       '9.0', '11.0', '12.0', '10.0', '13.0'], dtype=object)

In [1088]:
tbl_BP['ZONE'] = tbl_BP['ZONE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str)
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('102', '2')
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('108', '8')
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('104', '4')
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('44', '9')
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('46', '11')

C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\4054397438.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)


In [1089]:
#Drop duplicates based on ID column by only keeping the unique ID associated with the most recent Ship Date.  
#If there are rows where both tracking no and ship date are duplicated, only keep the first one.
tbl_BP = tbl_BP.sort_values('SHIP_DATE', ascending=False)
tbl_BP = tbl_BP.drop_duplicates(subset='TRACKING_NUMBER', keep='first')
tbl_BP = tbl_BP.drop_duplicates(subset='TRACKING_NUMBER', keep='first')
#tbl_BP.head()

# Format PLD

In [1090]:
#Skip this if LWH Columns exist already. This line of code needs improvement. 
#turn dimensions into L W H columns. no need to run this if they are already separate. 
tbl_BP[['L', 'W', 'H']] = tbl_BP['DIMENSIONS'].str.split('x', expand=True)

In [1091]:
#Skip this if LWH Columns are integers. 
tbl_BP[['L', 'W', 'H']] = tbl_BP[['L', 'W', 'H']].apply(pd.to_numeric)
tbl_BP['L'] = tbl_BP['L'].apply(lambda x: round(x,0) if x - math.floor(x) < 0.5 else np.ceil(x))
tbl_BP['W'] = tbl_BP['W'].apply(lambda x: round(x,0) if x - math.floor(x) < 0.5 else np.ceil(x))
tbl_BP['H'] = tbl_BP['H'].apply(lambda x: round(x,0) if x - math.floor(x) < 0.5 else np.ceil(x))


In [1092]:
# Ensure that the columns are of a numeric type (float)
tbl_BP['L'] = tbl_BP['L'].astype(float)
tbl_BP['W'] = tbl_BP['W'].astype(float)
tbl_BP['H'] = tbl_BP['H'].astype(float)

# Create a temporary DataFrame with the columns 'L', 'W', and 'H'
temp_df = tbl_BP[['L', 'W', 'H']]

# Sort each row in descending order and reverse the columns to maintain the order L, W, H
sorted_temp_df = np.sort(temp_df.values, axis=1)[:, ::-1]

# Assign the sorted values back to the original DataFrame
tbl_BP[['L', 'W', 'H']] = sorted_temp_df

In [1093]:
# Create columns of GIRTH_AND_L and CUBIC_INCH
tbl_BP['GIRTH_AND_L'] = round(tbl_BP['W'] * 2 + tbl_BP['H'] * 2 + tbl_BP['L'],0)
tbl_BP['CUBIC_INCH'] = round(tbl_BP['W'] * tbl_BP['H'] * tbl_BP['L'],0)


In [1094]:
# Create ACTUAL_WEIGHT_LB(rounded up to the nearest lb) and ACTUAL_WEIGHT_OZ(rounded up to the nearest oz). 
# Still keep the same value if the cell is empty string or has any missing values.  
tbl_BP['ACTUAL_WEIGHT_LB'] = tbl_BP['ACTUAL_WEIGHT'].apply(lambda x: np.ceil(x) if pd.notna(x) and x != '' else x)
tbl_BP['ACTUAL_WEIGHT_OZ_1'] = tbl_BP['ACTUAL_WEIGHT']*16
tbl_BP['ACTUAL_WEIGHT_OZ'] = tbl_BP['ACTUAL_WEIGHT_OZ_1'].apply(lambda x: np.ceil(x) if pd.notna(x) and x != '' else x)
tbl_BP = tbl_BP.drop('ACTUAL_WEIGHT_OZ_1', axis=1)

In [1095]:
# Skip this if the left 5 digits are provided already
# get the left 5 digits of the zip code or fill the zip to 5 digits
#import re
#split_col = tbl_BP['ZIP_CODE'].str.split('-')
#left_col = split_col.str[0]
#tbl_BP['ZIP_CODE'] = left_col
#tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].apply(lambda x: re.match(r'\d{5}', str(x)).group(0) if (pd.notna(x) and re.match(r'\d{5}', str(x))) else '')
# fill the 3 and 4 digits zip to 5 digits
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str)
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

# Step 3 - b. Rerate Begin
#====================UPS====================

#REDE (UPS NDA Early): commercial & residential, lb    

#RED (UPS NDA): commercial & residential, lb

#REDS (UPS NDA Saver): commercial & residential, lb

#2DAM (UPS 2DA A.M.): commercial & residential, lb

#BLUE (UPS 2DA): commercial & residential, lb

#ORNG (UPS 3DA): commercial & residential, lb

#GRND (UPS Ground Commercial): commercial only, lb

#GRES (UPS Ground Residential): residential only, lb

#SRPT<1 (UPS Surepost 1#>): commercial & residential, oz

#SRPT (UPS Surepost): commercial & residential, lb

#====================DHL====================

#DHLG (DHL SmartMail Parcel Plus Ground 1-25): commercial & residential, lb

#DHLG<1 (DHL SmartMail Parcel Ground < 1lb): commercial & residential, oz

#DHLE (DHL SmartMail Parcel Plus Expedited 1-25): commercial & residential, lb

#DHLE<1 (DHL SmartMail Parcel Expedited  < 1lb): commercial & residential, oz

#DHLEM (DHL SmartMail Parcel Expedited Max): commercial & residential, both oz and lb

#====================USPS====================

#USPSAG (USPS Auctane GRND): commercial & residential, lb

#USPSAP (USPS Auctane PM): commercial & residential, lb

#==========================MI===========================================

#MIPLE (MI Parcel Select Lightweight Expedited): commercial & residential, oz

#MIPH (MI Parcel Select Heavyweight): commercial & residential, lb

# ========= UPS Section ==========

In [982]:
# add a column named DAS_CATEGORY to PLD with ups accessorial category DAS, DASE, RA, HI, or AK. One package (or row) can only have one single category.
tbl_area_surcharge = pd.read_excel('C:\\Users\\sliu\\BD\\re-rate_model\\areaSurchargeZipsUs_2023.xlsx')
tbl_area_surcharge['Zip'] = tbl_area_surcharge['Zip'].astype(str)
tbl_area_surcharge['Zip'] = tbl_area_surcharge['Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(tbl_area_surcharge[["Zip", "Type"]], left_on='ZIP_CODE', right_on='Zip', how='left')
tbl_BP['DAS_CATEGORY'] = tbl_BP['Type']
tbl_BP = tbl_BP.drop('Type', axis=1)
tbl_BP = tbl_BP.drop('Zip', axis=1)
#tbl_BP.info()
#tbl_BP['ZONE'].unique()

# REDE (UPS NDA Early) 

In [983]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_1 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'REDE')]
tbl_PLD_original_REDE = tbl_BP.copy()

In [984]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_REDE['BILLED_WEIGHT_LB'] = tbl_PLD_original_REDE.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)


#add Freight column to PLD
tbl_PLD_original_REDE['ZONE'] = tbl_PLD_original_REDE['ZONE'].astype(str)
tbl_PLD_original_REDE = tbl_PLD_original_REDE.merge(tbl_clientRates_1[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_REDE = tbl_PLD_original_REDE.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_REDE = tbl_PLD_original_REDE.rename(columns={'CLIENT_RATES': 'FRT_REDE'})
   

#add Residential Surcharge column to PLD
tbl_PLD_original_REDE['RES_REDE'] = tbl_PLD_original_REDE.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_REDE['DAS_REDE'] = tbl_PLD_original_REDE.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))))), axis=1)




#add fuel surcharge % column to PLD
tbl_PLD_original_REDE['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_REDE['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_REDE['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')




#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_REDE = tbl_PLD_original_REDE.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_REDE.rename(columns={'Domestic Air': 'FSC%_REDE'}, inplace=True)







#add AHS column to PLD
tbl_PLD_original_REDE['ZONE'] = pd.to_numeric(tbl_PLD_original_REDE['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_REDE['AHS_1'] = tbl_PLD_original_REDE.apply(lambda row: 
                                                          AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_REDE['AHS_2'] = tbl_PLD_original_REDE.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_REDE['AHS_3'] = tbl_PLD_original_REDE.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_REDE['AHS_4'] = tbl_PLD_original_REDE.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_REDE['AHS_REDE'] = tbl_PLD_original_REDE['AHS_1'] + tbl_PLD_original_REDE['AHS_2'] + tbl_PLD_original_REDE['AHS_3'] + tbl_PLD_original_REDE['AHS_4']
tbl_PLD_original_REDE = tbl_PLD_original_REDE.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_REDE['TOTAL_REDE'] = round(tbl_PLD_original_REDE.apply(lambda row: 
                                                                  (1 + row['FSC%_REDE']) * (row['FRT_REDE'] + row['RES_REDE'] + row['DAS_REDE'] + row['AHS_REDE']) if not pd.isna(row['FRT_REDE']) and row['FRT_REDE'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_REDE.to_csv('test.csv')




# RED (UPS NDA)

In [985]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_2 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'RED')]
tbl_PLD_original_RED = tbl_BP.copy()

In [986]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_RED['BILLED_WEIGHT_LB'] = tbl_PLD_original_RED.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_RED['ZONE'] = tbl_PLD_original_RED['ZONE'].astype(str)
tbl_PLD_original_RED = tbl_PLD_original_RED.merge(tbl_clientRates_2[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_RED = tbl_PLD_original_RED.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_RED = tbl_PLD_original_RED.rename(columns={'CLIENT_RATES': 'FRT_RED'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_RED['RES_RED'] = tbl_PLD_original_RED.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_RED['DAS_RED'] = tbl_PLD_original_RED.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_RED['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_RED['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_RED['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_RED = tbl_PLD_original_RED.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_RED.rename(columns={'Domestic Air': 'FSC%_RED'}, inplace=True)


#add AHS column to PLD
tbl_PLD_original_RED['ZONE'] = pd.to_numeric(tbl_PLD_original_RED['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_RED['AHS_1'] = tbl_PLD_original_RED.apply(lambda row: 
                                                           AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_RED['AHS_2'] = tbl_PLD_original_RED.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_RED['AHS_3'] = tbl_PLD_original_RED.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_RED['AHS_4'] = tbl_PLD_original_RED.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_RED['AHS_RED'] = tbl_PLD_original_RED['AHS_1'] + tbl_PLD_original_RED['AHS_2'] + tbl_PLD_original_RED['AHS_3'] + tbl_PLD_original_RED['AHS_4']
tbl_PLD_original_RED = tbl_PLD_original_RED.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_RED['TOTAL_RED'] = round(tbl_PLD_original_RED.apply(lambda row: 
                                                                  (1 + row['FSC%_RED']) * (row['FRT_RED'] + row['RES_RED'] + row['DAS_RED'] + row['AHS_RED']) if not pd.isna(row['FRT_RED']) and row['FRT_RED'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_RED.to_csv('test.csv')

# REDS (UPS NDA Saver)

In [987]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_3 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'REDS')]
tbl_PLD_original_REDS = tbl_BP.copy()

In [988]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_REDS['BILLED_WEIGHT_LB'] = tbl_PLD_original_REDS.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_REDS['ZONE'] = tbl_PLD_original_REDS['ZONE'].astype(str)
tbl_PLD_original_REDS = tbl_PLD_original_REDS.merge(tbl_clientRates_3[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_REDS = tbl_PLD_original_REDS.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_REDS = tbl_PLD_original_REDS.rename(columns={'CLIENT_RATES': 'FRT_REDS'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_REDS['RES_REDS'] = tbl_PLD_original_REDS.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_REDS['DAS_REDS'] = tbl_PLD_original_REDS.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_REDS['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_REDS['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_REDS['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_REDS = tbl_PLD_original_REDS.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_REDS.rename(columns={'Domestic Air': 'FSC%_REDS'}, inplace=True)


#add AHS column to PLD
tbl_PLD_original_REDS['ZONE'] = pd.to_numeric(tbl_PLD_original_REDS['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_REDS['AHS_1'] = tbl_PLD_original_REDS.apply(lambda row: 
                                                           AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_REDS['AHS_2'] = tbl_PLD_original_REDS.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_REDS['AHS_3'] = tbl_PLD_original_REDS.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_REDS['AHS_4'] = tbl_PLD_original_REDS.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_REDS['AHS_REDS'] = tbl_PLD_original_REDS['AHS_1'] + tbl_PLD_original_REDS['AHS_2'] + tbl_PLD_original_REDS['AHS_3'] + tbl_PLD_original_REDS['AHS_4']
tbl_PLD_original_REDS = tbl_PLD_original_REDS.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_REDS['TOTAL_REDS'] = round(tbl_PLD_original_REDS.apply(lambda row: 
                                                                  (1 + row['FSC%_REDS']) * (row['FRT_REDS'] + row['RES_REDS'] + row['DAS_REDS'] + row['AHS_REDS']) if not pd.isna(row['FRT_REDS']) and row['FRT_REDS'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_REDS.to_csv('test.csv')

# 2DAM (UPS 2DA A.M.)

In [989]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_4 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == '2DAM')]
tbl_PLD_original_2DAM = tbl_BP.copy()

In [990]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_2DAM['BILLED_WEIGHT_LB'] = tbl_PLD_original_2DAM.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_2DAM['ZONE'] = tbl_PLD_original_2DAM['ZONE'].astype(str)
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.merge(tbl_clientRates_4[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.rename(columns={'CLIENT_RATES': 'FRT_2DAM'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_2DAM['RES_2DAM'] = tbl_PLD_original_2DAM.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_2DAM['DAS_2DAM'] = tbl_PLD_original_2DAM.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_2DAM['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_2DAM['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_2DAM['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_2DAM.rename(columns={'Domestic Air': 'FSC%_2DAM'}, inplace=True)


#add AHS column to PLD
tbl_PLD_original_2DAM['ZONE'] = pd.to_numeric(tbl_PLD_original_2DAM['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_2DAM['AHS_1'] = tbl_PLD_original_2DAM.apply(lambda row: 
                                                           AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_2DAM['AHS_2'] = tbl_PLD_original_2DAM.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_2DAM['AHS_3'] = tbl_PLD_original_2DAM.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_2DAM['AHS_4'] = tbl_PLD_original_2DAM.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_2DAM['AHS_2DAM'] = tbl_PLD_original_2DAM['AHS_1'] + tbl_PLD_original_2DAM['AHS_2'] + tbl_PLD_original_2DAM['AHS_3'] + tbl_PLD_original_2DAM['AHS_4']
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_2DAM['TOTAL_2DAM'] = round(tbl_PLD_original_2DAM.apply(lambda row: 
                                                                  (1 + row['FSC%_2DAM']) * (row['FRT_2DAM'] + row['RES_2DAM'] + row['DAS_2DAM'] + row['AHS_2DAM']) if not pd.isna(row['FRT_2DAM']) and row['FRT_2DAM'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_2DAM.to_csv('test.csv')

# BLUE (UPS 2DA)

In [991]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_5 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'BLUE')]
tbl_PLD_original_BLUE = tbl_BP.copy()

In [992]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_BLUE['BILLED_WEIGHT_LB'] = tbl_PLD_original_BLUE.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_BLUE['ZONE'] = tbl_PLD_original_BLUE['ZONE'].astype(str)
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.merge(tbl_clientRates_5[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.rename(columns={'CLIENT_RATES': 'FRT_BLUE'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_BLUE['RES_BLUE'] = tbl_PLD_original_BLUE.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_BLUE['DAS_BLUE'] = tbl_PLD_original_BLUE.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_BLUE['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_BLUE['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_BLUE['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')

df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date

#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_BLUE.rename(columns={'Domestic Air': 'FSC%_BLUE'}, inplace=True)




#add AHS column to PLD
tbl_PLD_original_BLUE['ZONE'] = pd.to_numeric(tbl_PLD_original_BLUE['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_BLUE['AHS_1'] = tbl_PLD_original_BLUE.apply(lambda row: 
                                                           AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_BLUE['AHS_2'] = tbl_PLD_original_BLUE.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_BLUE['AHS_3'] = tbl_PLD_original_BLUE.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_BLUE['AHS_4'] = tbl_PLD_original_BLUE.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_BLUE['AHS_BLUE'] = tbl_PLD_original_BLUE['AHS_1'] + tbl_PLD_original_BLUE['AHS_2'] + tbl_PLD_original_BLUE['AHS_3'] + tbl_PLD_original_BLUE['AHS_4']
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_BLUE['TOTAL_BLUE'] = round(tbl_PLD_original_BLUE.apply(lambda row: 
                                                                  (1 + row['FSC%_BLUE']) * (row['FRT_BLUE'] + row['RES_BLUE'] + row['DAS_BLUE'] + row['AHS_BLUE']) if not pd.isna(row['FRT_BLUE']) and row['FRT_BLUE'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_BLUE.to_csv('test.csv')

# ORNG (UPS 3DA)

In [994]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_6 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'ORNG')]
tbl_PLD_original_ORNG = tbl_BP.copy()

In [995]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_ORNG['BILLED_WEIGHT_LB'] = tbl_PLD_original_ORNG.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_ORNG['ZONE'] = tbl_PLD_original_ORNG['ZONE'].astype(str)
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.merge(tbl_clientRates_6[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.rename(columns={'CLIENT_RATES': 'FRT_ORNG'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_ORNG['RES_ORNG'] = tbl_PLD_original_ORNG.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_ORNG['DAS_ORNG'] = tbl_PLD_original_ORNG.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_ORNG['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_ORNG['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_ORNG['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_ORNG.rename(columns={'Domestic Air': 'FSC%_ORNG'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_ORNG['ZONE'] = pd.to_numeric(tbl_PLD_original_ORNG['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_ORNG['AHS_1'] = tbl_PLD_original_ORNG.apply(lambda row: 
                                                           AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_ORNG['AHS_2'] = tbl_PLD_original_ORNG.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_ORNG['AHS_3'] = tbl_PLD_original_ORNG.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_ORNG['AHS_4'] = tbl_PLD_original_ORNG.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_ORNG['AHS_ORNG'] = tbl_PLD_original_ORNG['AHS_1'] + tbl_PLD_original_ORNG['AHS_2'] + tbl_PLD_original_ORNG['AHS_3'] + tbl_PLD_original_ORNG['AHS_4']
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_ORNG['TOTAL_ORNG'] = round(tbl_PLD_original_ORNG.apply(lambda row: 
                                                                  (1 + row['FSC%_ORNG']) * (row['FRT_ORNG'] + row['RES_ORNG'] + row['DAS_ORNG'] + row['AHS_ORNG']) if not pd.isna(row['FRT_ORNG']) and row['FRT_ORNG'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_ORNG.to_csv('test.csv')

# GRND (UPS Ground Commercial)

In [996]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_7 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'GRND')]
tbl_PLD_original_GRND = tbl_BP[(tbl_BP['RESIDENTIAL_FLAG'] == False)].copy()

In [997]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_GRND['BILLED_WEIGHT_LB'] = tbl_PLD_original_GRND.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_GRND['ZONE'] = tbl_PLD_original_GRND['ZONE'].astype(str)
tbl_PLD_original_GRND = tbl_PLD_original_GRND.merge(tbl_clientRates_7[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_GRND = tbl_PLD_original_GRND.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_GRND = tbl_PLD_original_GRND.rename(columns={'CLIENT_RATES': 'FRT_GRND'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_GRND['RES_GRND'] = tbl_PLD_original_GRND.apply(lambda row: Resi_Surcharge_Ground if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_GRND['DAS_GRND'] = tbl_PLD_original_GRND.apply(lambda row: 
                                                            DAS_Comm_Ground if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Ground if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Ground if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Ground if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_GRND['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_GRND['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_GRND['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Ground column as FSC. 
tbl_PLD_original_GRND = tbl_PLD_original_GRND.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air'], axis=1)
tbl_PLD_original_GRND.rename(columns={'Ground': 'FSC%_GRND'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_GRND['ZONE'] = pd.to_numeric(tbl_PLD_original_GRND['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_GRND['AHS_1'] = tbl_PLD_original_GRND.apply(lambda row: 
                                                           AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_GRND['AHS_2'] = tbl_PLD_original_GRND.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_GRND['AHS_3'] = tbl_PLD_original_GRND.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_GRND['AHS_4'] = tbl_PLD_original_GRND.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_GRND['AHS_GRND'] = tbl_PLD_original_GRND['AHS_1'] + tbl_PLD_original_GRND['AHS_2'] + tbl_PLD_original_GRND['AHS_3'] + tbl_PLD_original_GRND['AHS_4']
tbl_PLD_original_GRND = tbl_PLD_original_GRND.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_GRND['TOTAL_GRND'] = round(tbl_PLD_original_GRND.apply(lambda row: 
                                                                  (1 + row['FSC%_GRND']) * (row['FRT_GRND'] + row['RES_GRND'] + row['DAS_GRND'] + row['AHS_GRND']) if not pd.isna(row['FRT_GRND']) and row['FRT_GRND'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_GRND.to_csv('test.csv')

# GRES (UPS Ground Residential)

In [998]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_8 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'GRES')]
tbl_PLD_original_GRES = tbl_BP[(tbl_BP['RESIDENTIAL_FLAG'] == True)].copy()

In [999]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_GRES['BILLED_WEIGHT_LB'] = tbl_PLD_original_GRES.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_GRES['ZONE'] = tbl_PLD_original_GRES['ZONE'].astype(str)
tbl_PLD_original_GRES = tbl_PLD_original_GRES.merge(tbl_clientRates_8[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_GRES = tbl_PLD_original_GRES.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_GRES = tbl_PLD_original_GRES.rename(columns={'CLIENT_RATES': 'FRT_GRES'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_GRES['RES_GRES'] = tbl_PLD_original_GRES.apply(lambda row: Resi_Surcharge_Ground if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_GRES['DAS_GRES'] = tbl_PLD_original_GRES.apply(lambda row: 
                                                            DAS_Comm_Ground if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Ground if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Ground if (row['DAS_CATEGORY'] == 'DAS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Ground if (row['DAS_CATEGORY'] == 'DASE' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_GRES['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_GRES['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_GRES['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Ground column as FSC. 
tbl_PLD_original_GRES = tbl_PLD_original_GRES.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air'], axis=1)
tbl_PLD_original_GRES.rename(columns={'Ground': 'FSC%_GRES'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_GRES['ZONE'] = pd.to_numeric(tbl_PLD_original_GRES['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_GRES['AHS_1'] = tbl_PLD_original_GRES.apply(lambda row: 
                                                           AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_GRES['AHS_2'] = tbl_PLD_original_GRES.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_GRES['AHS_3'] = tbl_PLD_original_GRES.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_GRES['AHS_4'] = tbl_PLD_original_GRES.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_GRES['AHS_GRES'] = tbl_PLD_original_GRES['AHS_1'] + tbl_PLD_original_GRES['AHS_2'] + tbl_PLD_original_GRES['AHS_3'] + tbl_PLD_original_GRES['AHS_4']
tbl_PLD_original_GRES = tbl_PLD_original_GRES.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_GRES['TOTAL_GRES'] = round(tbl_PLD_original_GRES.apply(lambda row: 
                                                                  (1 + row['FSC%_GRES']) * (row['FRT_GRES'] + row['RES_GRES'] + row['DAS_GRES'] + row['AHS_GRES']) if not pd.isna(row['FRT_GRES']) and row['FRT_GRES'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_GRES.to_csv('test.csv')

# SRPT<1 (UPS Surepost 1#>)

In [1000]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_9 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'SRPT<1')]
tbl_PLD_original_SRPT1 = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] < 1)].copy()

In [1001]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_SRPT1['BILLED_WEIGHT_OZ'] = tbl_PLD_original_SRPT1.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups_srpt))*16 if (row['CUBIC_INCH'] >= 3456) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)


#add Freight column to PLD
tbl_PLD_original_SRPT1['ZONE'] = tbl_PLD_original_SRPT1['ZONE'].astype(str)
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.merge(tbl_clientRates_9[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.rename(columns={'CLIENT_RATES': 'FRT_SRPT1'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_SRPT1['RES_SRPT1'] = tbl_PLD_original_SRPT1.apply(lambda row: 0 if row['RESIDENTIAL_FLAG'] == True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_SRPT1['DAS_SRPT1'] = tbl_PLD_original_SRPT1.apply(lambda row: 
                                                            DAS_SurePost if (row['DAS_CATEGORY'] == 'DAS') else 
                                                           (DAS_Extended_SurePost if (row['DAS_CATEGORY'] == 'DASE') else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))), axis=1)



#add fuel surcharge % column to PLD
tbl_PLD_original_SRPT1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_SRPT1['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_SRPT1['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Ground column as FSC. 
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air'], axis=1)
tbl_PLD_original_SRPT1.rename(columns={'Ground': 'FSC%_SRPT1'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_SRPT1['ZONE'] = pd.to_numeric(tbl_PLD_original_SRPT1['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_SRPT1['AHS_1'] = tbl_PLD_original_SRPT1.apply(lambda row: 
                                                           AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_SRPT1['AHS_2'] = tbl_PLD_original_SRPT1.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_SRPT1['AHS_3'] = tbl_PLD_original_SRPT1.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_SRPT1['AHS_4'] = tbl_PLD_original_SRPT1.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_SRPT1['AHS_SRPT1'] = tbl_PLD_original_SRPT1['AHS_1'] + tbl_PLD_original_SRPT1['AHS_2'] + tbl_PLD_original_SRPT1['AHS_3'] + tbl_PLD_original_SRPT1['AHS_4']
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_SRPT1['TOTAL_SRPT1'] = round(tbl_PLD_original_SRPT1.apply(lambda row: 
                                                                  (1 + row['FSC%_SRPT1']) * (row['FRT_SRPT1'] + row['RES_SRPT1'] + row['DAS_SRPT1'] + row['AHS_SRPT1']) if not pd.isna(row['FRT_SRPT1']) and row['FRT_SRPT1'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_SRPT1.to_csv('test.csv')

# SRPT (UPS Surepost)

In [1002]:
#Get the right client rate and the corresponding PLD
#get client rate and a copy of PLD
tbl_clientRates_10 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'SRPT')]
tbl_PLD_original_SRPT = tbl_BP.copy()

In [1003]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_SRPT['BILLED_WEIGHT_LB'] = tbl_PLD_original_SRPT.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups_srpt)) if (row['CUBIC_INCH'] >= 3456) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)




#add Freight column to PLD
tbl_PLD_original_SRPT['ZONE'] = tbl_PLD_original_SRPT['ZONE'].astype(str)
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.merge(tbl_clientRates_10[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.rename(columns={'CLIENT_RATES': 'FRT_SRPT'})
   

#add Residential Surcharge column to PLD
tbl_PLD_original_SRPT['RES_SRPT'] = tbl_PLD_original_SRPT.apply(lambda row: 0 if row['RESIDENTIAL_FLAG'] == True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_SRPT['DAS_SRPT'] = tbl_PLD_original_SRPT.apply(lambda row: 
                                                            DAS_SurePost if (row['DAS_CATEGORY'] == 'DAS') else 
                                                           (DAS_Extended_SurePost if (row['DAS_CATEGORY'] == 'DASE') else 
                                                           (Remote_Area if (row['DAS_CATEGORY'] == 'RA') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY'] == 'AK') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY'] == 'HI') else 0)))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_SRPT['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_SRPT['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_SRPT['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Ground column as FSC. 
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air'], axis=1)
tbl_PLD_original_SRPT.rename(columns={'Ground': 'FSC%_SRPT'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_SRPT['ZONE'] = pd.to_numeric(tbl_PLD_original_SRPT['ZONE'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_SRPT['AHS_1'] = tbl_PLD_original_SRPT.apply(lambda row: 
                                                           AH_Girth_2 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] == 2) else 
                                                           (AH_Girth_3_4 if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 2 and row['ZONE'] < 5) else
                                                           (AH_Girth_5_up if (row['GIRTH_AND_L'] > 105 and row['ZONE'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_SRPT['AHS_2'] = tbl_PLD_original_SRPT.apply(lambda row: 
                                                           AH_L_2 if (row['L'] > 48 and row['ZONE'] == 2) else 
                                                           (AH_L_3_4 if (row['L'] > 48 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_L_5_up if (row['L'] > 48 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_SRPT['AHS_3'] = tbl_PLD_original_SRPT.apply(lambda row:
                                                           AH_W_2 if (row['W'] > 30 and row['ZONE'] == 2) else 
                                                           (AH_W_3_4 if (row['W'] > 30 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_W_5_up if (row['W'] > 30 and row['ZONE'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_SRPT['AHS_4'] = tbl_PLD_original_SRPT.apply(lambda row:
                                                           AH_WGT_2 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] == 2) else 
                                                           (AH_WGT_3_4 if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 2 and row['ZONE'] < 5) else 
                                                           (AH_WGT_5_up if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE'] > 4) else 0)), axis=1)

tbl_PLD_original_SRPT['AHS_SRPT'] = tbl_PLD_original_SRPT['AHS_1'] + tbl_PLD_original_SRPT['AHS_2'] + tbl_PLD_original_SRPT['AHS_3'] + tbl_PLD_original_SRPT['AHS_4']
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_SRPT['TOTAL_SRPT'] = round(tbl_PLD_original_SRPT.apply(lambda row: 
                                                                  (1 + row['FSC%_SRPT']) * (row['FRT_SRPT'] + row['RES_SRPT'] + row['DAS_SRPT'] + row['AHS_SRPT']) if not pd.isna(row['FRT_SRPT']) and row['FRT_SRPT'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_SRPT.to_csv('test.csv')

# DHLG (DHL SmartMail Parcel Plus Ground)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound  
5123556: Montebello -> 90640


# DHLG

In [1004]:
styled_html = HTML("""
<p style="font-family: 'Arial'; font-size: 20px; color: red; background-color: yellow;">
    Edit Location 3 of 8 (Only if you use DHL, default facility is MS3)
</p>
""")

display(styled_html)

In [1005]:
#select the rate to use, #Decide the part of PLD to rerate
tbl_clientRates_11 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLG_5122739')] # update the number here. 
tbl_PLD_original_DHLG = tbl_BP.copy()

In [1006]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLG['BILLED_WEIGHT_LB'] = tbl_PLD_original_DHLG.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

#add Freight column to PLD
tbl_PLD_original_DHLG['ZONE'] = tbl_PLD_original_DHLG['ZONE'].astype(str)
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.merge(tbl_clientRates_11[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.rename(columns={'CLIENT_RATES': 'FRT_DHLG'})


#add fuel surcharge % column to PLD
tbl_PLD_original_DHLG['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLG['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLG['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()



# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLG.rename(columns={'FSC': 'FSC%_DHLG'}, inplace=True)
tbl_PLD_original_DHLG['FSC_DHLG'] = tbl_PLD_original_DHLG.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLG'] if (row['ACTUAL_WEIGHT'] <= 1) else
                                                                          (row['BILLED_WEIGHT_LB']*row['FSC%_DHLG']), axis=1)


#add NQD Surcharge column to PLD
tbl_PLD_original_DHLG['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLG['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim
tbl_PLD_original_DHLG['NQD_DHLG_1'] = tbl_PLD_original_DHLG.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)

#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this)
tbl_PLD_original_DHLG['NQD_DHLG_2'] = tbl_PLD_original_DHLG.apply(lambda row:
                                                           np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)

#longest side_1
tbl_PLD_original_DHLG['NQD_DHLG_3'] = tbl_PLD_original_DHLG.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLG['NQD_DHLG_4'] = tbl_PLD_original_DHLG.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLG['NQD_DHLG_5'] = tbl_PLD_original_DHLG.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLG['NQD_DHLG_OTHER'] = tbl_PLD_original_DHLG['NQD_DHLG_1'] + tbl_PLD_original_DHLG['NQD_DHLG_3'] + tbl_PLD_original_DHLG['NQD_DHLG_4'] + tbl_PLD_original_DHLG['NQD_DHLG_5']
tbl_PLD_original_DHLG['NQD_DHLG_GIRTH_AND_L'] = tbl_PLD_original_DHLG['NQD_DHLG_2']
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.drop(['NQD_DHLG_1','NQD_DHLG_2','NQD_DHLG_3','NQD_DHLG_4','NQD_DHLG_5'], axis=1)


#Get Total
tbl_PLD_original_DHLG['TOTAL_DHLG'] = round(tbl_PLD_original_DHLG.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLG'] == '') | (pd.isna(row['FRT_DHLG'])) | (row['NQD_DHLG_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLG_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLG'] + (row['FSC_DHLG'] + row['NQD_DHLG_OTHER'] + row['NQD_DHLG_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)


#tbl_PLD_original_DHLG.to_csv('test.csv')


# DHLG<1 (DHL SmartMail Parcel Ground)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound

# DHLG<1

In [1007]:
styled_html = HTML("""
<p style="font-family: 'Arial'; font-size: 20px; color: red; background-color: yellow;">
    Edit Location 4 of 8 (Only if you use DHL, default facility is MS3)
</p>
""")

display(styled_html)

In [1008]:
#select the rate to use, #Decide the part of PLD to rerate
tbl_clientRates_12 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLG<1_5122739')] # Edit the number here. 
tbl_PLD_original_DHLG1 = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] < 1)].copy()

In [1009]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLG1['BILLED_WEIGHT_OZ'] = tbl_PLD_original_DHLG1.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl))*16 if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)


#add Freight column to PLD
tbl_PLD_original_DHLG1['ZONE'] = tbl_PLD_original_DHLG1['ZONE'].astype(str)
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.merge(tbl_clientRates_12[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.rename(columns={'CLIENT_RATES': 'FRT_DHLG1'})

#add fuel surcharge % column to PLD
tbl_PLD_original_DHLG1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLG1['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
# when actual weight < 1, use actual weight non round up to get FSC, otherwise, use billed weight to get FSC
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLG1['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLG1.rename(columns={'FSC': 'FSC%_DHLG1'}, inplace=True)
tbl_PLD_original_DHLG1['FSC_DHLG1'] = tbl_PLD_original_DHLG1.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLG1'], axis=1)




#add NQD Surcharge column to PLD
tbl_PLD_original_DHLG1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLG1['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim, billed weight is used to determine NQD, there is no <1 lb here
tbl_PLD_original_DHLG1['NQD_DHLG1_1'] = tbl_PLD_original_DHLG1.apply(lambda row: 
                                                            2.5 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this, so we make it null)
#1 over 1lbs, under lbs
#and those < 1 lb will use lb to determine NQD
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16)
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use actual weight
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight
tbl_PLD_original_DHLG1['NQD_DHLG1_2'] = tbl_PLD_original_DHLG1.apply(lambda row:
                                                            np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)

#longest side_1
tbl_PLD_original_DHLG1['NQD_DHLG1_3'] = tbl_PLD_original_DHLG1.apply(lambda row: 
                                                            2.5 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLG1['NQD_DHLG1_4'] = tbl_PLD_original_DHLG1.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLG1['NQD_DHLG1_5'] = tbl_PLD_original_DHLG1.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLG1['NQD_DHLG1_OTHER'] = tbl_PLD_original_DHLG1['NQD_DHLG1_1'] + tbl_PLD_original_DHLG1['NQD_DHLG1_3'] + tbl_PLD_original_DHLG1['NQD_DHLG1_4'] + tbl_PLD_original_DHLG1['NQD_DHLG1_5']
tbl_PLD_original_DHLG1['NQD_DHLG1_GIRTH_AND_L'] = tbl_PLD_original_DHLG1['NQD_DHLG1_2']
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.drop(['NQD_DHLG1_1','NQD_DHLG1_2','NQD_DHLG1_3','NQD_DHLG1_4','NQD_DHLG1_5'], axis=1)


#Get Total
tbl_PLD_original_DHLG1['TOTAL_DHLG1'] = round(tbl_PLD_original_DHLG1.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLG1'] == '') | (pd.isna(row['FRT_DHLG1'])) | (row['NQD_DHLG1_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLG1_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLG1'] + (row['FSC_DHLG1'] + row['NQD_DHLG1_OTHER'] + row['NQD_DHLG1_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)

#tbl_PLD_original_DHLG1.to_csv('test.csv')

# DHLE (DHL SmartMail Parcel Plus Expedited)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound

In [1010]:
styled_html = HTML("""
<p style="font-family: 'Arial'; font-size: 20px; color: red; background-color: yellow;">
    Edit Location 5 of 8 (Only if you use DHL, default facility is MS3)
</p>
""")

display(styled_html)

In [1011]:

#get client rate and a copy of PLD
tbl_clientRates_13 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLE_5122739')] # Edit number here
tbl_PLD_original_DHLE = tbl_BP.copy()

In [1012]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLE['BILLED_WEIGHT_LB'] = tbl_PLD_original_DHLE.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

#add Freight column to PLD
tbl_PLD_original_DHLE['ZONE'] = tbl_PLD_original_DHLE['ZONE'].astype(str)
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.merge(tbl_clientRates_13[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.rename(columns={'CLIENT_RATES': 'FRT_DHLE'})



#add fuel surcharge % column to PLD
tbl_PLD_original_DHLE['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLE['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLE['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLE.rename(columns={'FSC': 'FSC%_DHLE'}, inplace=True)
tbl_PLD_original_DHLE['FSC_DHLE'] = tbl_PLD_original_DHLE.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLE'] if (row['ACTUAL_WEIGHT'] <= 1) else
                                                                          (row['BILLED_WEIGHT_LB']*row['FSC%_DHLE']), axis=1)


#add NQD Surcharge column to PLD
tbl_PLD_original_DHLE['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLE['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim
tbl_PLD_original_DHLE['NQD_DHLE_1'] = tbl_PLD_original_DHLE.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this)
tbl_PLD_original_DHLE['NQD_DHLE_2'] = tbl_PLD_original_DHLE.apply(lambda row:
                                                            np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)

#longest side_1
tbl_PLD_original_DHLE['NQD_DHLE_3'] = tbl_PLD_original_DHLE.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLE['NQD_DHLE_4'] = tbl_PLD_original_DHLE.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLE['NQD_DHLE_5'] = tbl_PLD_original_DHLE.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLE['NQD_DHLE_OTHER'] = tbl_PLD_original_DHLE['NQD_DHLE_1'] + tbl_PLD_original_DHLE['NQD_DHLE_3'] + tbl_PLD_original_DHLE['NQD_DHLE_4'] + tbl_PLD_original_DHLE['NQD_DHLE_5']
tbl_PLD_original_DHLE['NQD_DHLE_GIRTH_AND_L'] = tbl_PLD_original_DHLE['NQD_DHLE_2']
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.drop(['NQD_DHLE_1','NQD_DHLE_2','NQD_DHLE_3','NQD_DHLE_4','NQD_DHLE_5'], axis=1)


#Get Total
# note that for those with non-round up actual weight, if it is < 1, we still assign it the freight rate. 

tbl_PLD_original_DHLE['TOTAL_DHLE'] = round(tbl_PLD_original_DHLE.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLE'] == '') | (pd.isna(row['FRT_DHLE'])) | (row['NQD_DHLE_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLE_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLE'] + (row['FSC_DHLE'] + row['NQD_DHLE_OTHER'] + row['NQD_DHLE_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)

#tbl_PLD_original_DHLE.to_csv('test.csv')

# DHLE<1 (DHL SmartMail Parcel Expedited)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound

# DHLE<1

In [1013]:
styled_html = HTML("""
<p style="font-family: 'Arial'; font-size: 20px; color: red; background-color: yellow;">
    Edit Location 6 of 8 (Only if you use DHL, default facility is MS3)
</p>
""")

display(styled_html)

In [1014]:
#Select the rate to use, #Decide the part of PLD to rerate
#get client rate and a copy of PLD
tbl_clientRates_14 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLE<1_5122739')] # Edit the number here. 
tbl_PLD_original_DHLE1 = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] < 1)].copy()

In [1015]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLE1['BILLED_WEIGHT_OZ'] = tbl_PLD_original_DHLE1.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl))*16 if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)


#add Freight column to PLD
tbl_PLD_original_DHLE1['ZONE'] = tbl_PLD_original_DHLE1['ZONE'].astype(str)
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.merge(tbl_clientRates_14[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.rename(columns={'CLIENT_RATES': 'FRT_DHLE1'})


#add fuel surcharge % column to PLD
tbl_PLD_original_DHLE1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLE1['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
# when actual weight < 1, use actual weight non round up to get FSC, otherwise, use billed weight to get FSC
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLE1['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLE1.rename(columns={'FSC': 'FSC%_DHLE1'}, inplace=True)
tbl_PLD_original_DHLE1['FSC_DHLE1'] = tbl_PLD_original_DHLE1.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLE1'], axis=1)


#add NQD Surcharge column to PLD
tbl_PLD_original_DHLE1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLE1['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim, billed weight is used to determine NQD, there is no <1 lb here
tbl_PLD_original_DHLE1['NQD_DHLE1_1'] = tbl_PLD_original_DHLE1.apply(lambda row: 
                                                            2.5 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this, so we make it null)
#1 over 1lbs, under lbs
#and those < 1 lb will use lb to determine NQD
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16)
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use actual weight
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight
tbl_PLD_original_DHLE1['NQD_DHLE1_2'] = tbl_PLD_original_DHLE1.apply(lambda row:
                                                            np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)

#longest side_1
tbl_PLD_original_DHLE1['NQD_DHLE1_3'] = tbl_PLD_original_DHLE1.apply(lambda row: 
                                                            2.5 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLE1['NQD_DHLE1_4'] = tbl_PLD_original_DHLE1.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLE1['NQD_DHLE1_5'] = tbl_PLD_original_DHLE1.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLE1['NQD_DHLE1_OTHER'] = tbl_PLD_original_DHLE1['NQD_DHLE1_1'] + tbl_PLD_original_DHLE1['NQD_DHLE1_3'] + tbl_PLD_original_DHLE1['NQD_DHLE1_4'] + tbl_PLD_original_DHLE1['NQD_DHLE1_5']
tbl_PLD_original_DHLE1['NQD_DHLE1_GIRTH_AND_L'] = tbl_PLD_original_DHLE1['NQD_DHLE1_2']
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.drop(['NQD_DHLE1_1','NQD_DHLE1_2','NQD_DHLE1_3','NQD_DHLE1_4','NQD_DHLE1_5'], axis=1)


#Get Total
tbl_PLD_original_DHLE1['TOTAL_DHLE1'] = round(tbl_PLD_original_DHLE1.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLE1'] == '') | (pd.isna(row['FRT_DHLE1'])) | (row['NQD_DHLE1_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLE1_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLE1'] + (row['FSC_DHLE1'] + row['NQD_DHLE1_OTHER'] + row['NQD_DHLE1_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)

#tbl_PLD_original_DHLE1.to_csv('test.csv')

# DHLEM (DHL SmartMail Parcel Expedited Max)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound

# DHLEM

In [1016]:
styled_html = HTML("""
<p style="font-family: 'Arial'; font-size: 20px; color: red; background-color: yellow;">
    Edit Location 7 of 8 (Only if you use DHL, default facility is MS3)
</p>
""")

display(styled_html)

In [1017]:
#Select the rate to use, #Decide the part of PLD to rerate
#get client rate and a copy of PLD
tbl_clientRates_15 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLEM_5122739')] # Edit number here
tbl_PLD_original_DHLEM1 = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] <= 1)].copy()


In [1018]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLEM1['BILLED_WEIGHT_OZ'] = tbl_PLD_original_DHLEM1.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl))*16 if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)

#add Freight column to PLD
tbl_PLD_original_DHLEM1['ZONE'] = tbl_PLD_original_DHLEM1['ZONE'].astype(str)
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.merge(tbl_clientRates_15[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.rename(columns={'CLIENT_RATES': 'FRT_DHLEM'})


#add fuel surcharge % column to PLD
tbl_PLD_original_DHLEM1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLEM1['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
# when actual weight < 1, use actual weight non round up to get FSC, otherwise, use billed weight to get FSC
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLEM1['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLEM1.rename(columns={'FSC': 'FSC%_DHLEM'}, inplace=True)
tbl_PLD_original_DHLEM1['FSC_DHLEM'] = tbl_PLD_original_DHLEM1.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLEM'], axis=1)




#add NQD Surcharge column to PLD
tbl_PLD_original_DHLEM1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLEM1['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim, billed weight is used to determine NQD, there is no <1 lb here
tbl_PLD_original_DHLEM1['NQD_DHLEM_1'] = tbl_PLD_original_DHLEM1.apply(lambda row: 
                                                            2.5 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this, so we make it null)
#1 over 1lbs, under lbs
#and those < 1 lb will use lb to determine NQD
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16)
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use actual weight
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight
tbl_PLD_original_DHLEM1['NQD_DHLEM_2'] = tbl_PLD_original_DHLEM1.apply(lambda row:
                                                            np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)

#longest side_1
tbl_PLD_original_DHLEM1['NQD_DHLEM_3'] = tbl_PLD_original_DHLEM1.apply(lambda row: 
                                                            2.5 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLEM1['NQD_DHLEM_4'] = tbl_PLD_original_DHLEM1.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLEM1['NQD_DHLEM_5'] = tbl_PLD_original_DHLEM1.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLEM1['NQD_DHLEM_OTHER'] = tbl_PLD_original_DHLEM1['NQD_DHLEM_1'] + tbl_PLD_original_DHLEM1['NQD_DHLEM_3'] + tbl_PLD_original_DHLEM1['NQD_DHLEM_4'] + tbl_PLD_original_DHLEM1['NQD_DHLEM_5']
tbl_PLD_original_DHLEM1['NQD_DHLEM_GIRTH_AND_L'] = tbl_PLD_original_DHLEM1['NQD_DHLEM_2']
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.drop(['NQD_DHLEM_1','NQD_DHLEM_2','NQD_DHLEM_3','NQD_DHLEM_4','NQD_DHLEM_5'], axis=1)


#Get Total
tbl_PLD_original_DHLEM1['TOTAL_DHLEM'] = round(tbl_PLD_original_DHLEM1.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLEM'] == '') | (pd.isna(row['FRT_DHLEM'])) | (row['NQD_DHLEM_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLEM_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLEM'] + (row['FSC_DHLEM'] + row['NQD_DHLEM_OTHER'] + row['NQD_DHLEM_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)


In [1019]:
styled_html = HTML("""
<p style="font-family: 'Arial'; font-size: 20px; color: red; background-color: yellow;">
    Edit Location 8 of 8 (Only if you use DHL, default facility is MS3)
</p>
""")

display(styled_html)

In [1020]:
#Select the rate to use, #Decide the part of PLD to rerate
#get client rate and a copy of PLD
tbl_clientRates_16 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLEM_5122739')] # Edit the number here
tbl_PLD_original_DHLEM = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] > 1)].copy()


In [1021]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLEM['BILLED_WEIGHT_LB'] = tbl_PLD_original_DHLEM.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

#add Freight column to PLD
tbl_PLD_original_DHLEM['ZONE'] = tbl_PLD_original_DHLEM['ZONE'].astype(str)
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.merge(tbl_clientRates_16[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.rename(columns={'CLIENT_RATES': 'FRT_DHLEM'})



#add fuel surcharge % column to PLD
tbl_PLD_original_DHLEM['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLEM['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLEM['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLEM.rename(columns={'FSC': 'FSC%_DHLEM'}, inplace=True)
tbl_PLD_original_DHLEM['FSC_DHLEM'] = tbl_PLD_original_DHLEM.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLEM'] if (row['ACTUAL_WEIGHT'] <= 1) else
                                                                          (row['BILLED_WEIGHT_LB']*row['FSC%_DHLEM']), axis=1)




#add NQD Surcharge column to PLD
tbl_PLD_original_DHLEM['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLEM['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim
tbl_PLD_original_DHLEM['NQD_DHLEM_1'] = tbl_PLD_original_DHLEM.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this)
tbl_PLD_original_DHLEM['NQD_DHLEM_2'] = tbl_PLD_original_DHLEM.apply(lambda row:
                                                           np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)

#longest side_1
tbl_PLD_original_DHLEM['NQD_DHLEM_3'] = tbl_PLD_original_DHLEM.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLEM['NQD_DHLEM_4'] = tbl_PLD_original_DHLEM.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLEM['NQD_DHLEM_5'] = tbl_PLD_original_DHLEM.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLEM['NQD_DHLEM_OTHER'] = tbl_PLD_original_DHLEM['NQD_DHLEM_1'] + tbl_PLD_original_DHLEM['NQD_DHLEM_3'] + tbl_PLD_original_DHLEM['NQD_DHLEM_4'] + tbl_PLD_original_DHLEM['NQD_DHLEM_5']
tbl_PLD_original_DHLEM['NQD_DHLEM_GIRTH_AND_L'] = tbl_PLD_original_DHLEM['NQD_DHLEM_2']
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.drop(['NQD_DHLEM_1','NQD_DHLEM_2','NQD_DHLEM_3','NQD_DHLEM_4','NQD_DHLEM_5'], axis=1)


#Get Total
# note that for those with non-round up actual weight, if it is < 1, we still assign it the freight rate. 

tbl_PLD_original_DHLEM['TOTAL_DHLEM'] = round(tbl_PLD_original_DHLEM.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLEM'] == '') | (pd.isna(row['FRT_DHLEM'])) | (row['NQD_DHLEM_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLEM_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLEM'] + (row['FSC_DHLEM'] + row['NQD_DHLEM_OTHER'] + row['NQD_DHLEM_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)



In [1022]:
tbl_PLD_original_DHLEM = pd.concat([tbl_PLD_original_DHLEM1, tbl_PLD_original_DHLEM], axis=0)
#tbl_PLD_original_DHLEM.to_csv('test.csv')

# USPSAG

In [1096]:
#Get the right client rate and the corresponding PLD
#get client rate and a copy of PLD
tbl_clientRates_17 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'USPSAG')]
tbl_PLD_original_USPSAG = tbl_BP.copy()

In [1097]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_USPSAG['BILLED_WEIGHT_LB'] = tbl_PLD_original_USPSAG.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_usps)) if (row['CUBIC_INCH'] > 1728) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

tbl_PLD_original_USPSAG['BILLED_WEIGHT_OZ'] = tbl_PLD_original_USPSAG.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_usps))*16 if (row['CUBIC_INCH'] > 1728) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)

tbl_PLD_original_USPSAG['BILLED_WEIGHT_LB'] = tbl_PLD_original_USPSAG.apply(lambda row:
                                                            row['BILLED_WEIGHT_OZ']/16 if (row['BILLED_WEIGHT_OZ'] <= 16) else 
                                                           (row['BILLED_WEIGHT_LB']), axis=1)

#usps will only rate for those not exceeding 70 lbs
tbl_PLD_original_USPSAG = tbl_PLD_original_USPSAG[(tbl_PLD_original_USPSAG['BILLED_WEIGHT_LB'] <= 70)]


#add Freight column to PLD
tbl_PLD_original_USPSAG_3 = tbl_PLD_original_USPSAG[(tbl_PLD_original_USPSAG['GIRTH_AND_L'] > 108) & (tbl_PLD_original_USPSAG['GIRTH_AND_L'] <= 130)]
tbl_PLD_original_USPSAG_none3 = tbl_PLD_original_USPSAG[(tbl_PLD_original_USPSAG['GIRTH_AND_L'] <= 108)]

#main case
tbl_PLD_original_USPSAG_none3['ZONE'] = tbl_PLD_original_USPSAG_none3['ZONE'].astype(str)
tbl_PLD_original_USPSAG_none3 = tbl_PLD_original_USPSAG_none3.merge(tbl_clientRates_17[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_USPSAG_none3 = tbl_PLD_original_USPSAG_none3.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_USPSAG_none3 = tbl_PLD_original_USPSAG_none3.rename(columns={'CLIENT_RATES': 'FRT_USPSAG'})

#3. for parcels that measure in combined length and girth more than 108 inches but not more than 130 inches, use oversized prices, 
#regardless of weight, based on the applicable zone.

tbl_oversize = tbl_clientRates_17[(tbl_clientRates_17['WEIGHT_LB'] == 9999)]
tbl_PLD_original_USPSAG_3['ZONE'] = tbl_PLD_original_USPSAG_3['ZONE'].astype(str)
tbl_PLD_original_USPSAG_3 = tbl_PLD_original_USPSAG_3.merge(tbl_oversize[["ZONE", "CLIENT_RATES"]], left_on=['ZONE'], right_on=['ZONE'], how='left')
tbl_PLD_original_USPSAG_3 = tbl_PLD_original_USPSAG_3.rename(columns={'CLIENT_RATES': 'FRT_USPSAG'})

#combine again
tbl_PLD_original_USPSAG = pd.concat([tbl_PLD_original_USPSAG_3, tbl_PLD_original_USPSAG_none3], axis=0)



C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\481323264.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_PLD_original_USPSAG_none3['ZONE'] = tbl_PLD_original_USPSAG_none3['ZONE'].astype(str)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\481323264.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_PLD_original_USPSAG_3['ZONE'] = tbl_PLD_original_USPSAG_3['ZONE'].astype(str)


In [1098]:
#6. Parcels that exceed 22 inches but not greater than 30 inches in length, add $4 per package. 
#7. parcels that exceed 30 inches in length, add $7 per package. 

tbl_PLD_original_USPSAG['EXTRA_USPSAG_1'] = tbl_PLD_original_USPSAG.apply(lambda row:
                                                            4 if (row['L'] > 22 and row['L'] <= 30) else 
                                                           (7 if (row['L'] > 30) else
                                                           (0)), axis=1)


#8. parcels that exceed 2 cubic feet (3456 cubic inches), add $15 per package. 
tbl_PLD_original_USPSAG['EXTRA_USPSAG_2'] = tbl_PLD_original_USPSAG.apply(lambda row:
                                                            15 if (row['CUBIC_INCH'] > 3456) else (0), axis=1)

#Get Total
# note that for those with non-round up actual weight, if it is < 1, we still assign it the freight rate. 

#Get Total, revise the logic that if the freight rate is 0 or some rate is 0, the total will be 0. ups can handle everything. 
tbl_PLD_original_USPSAG['TOTAL_USPSAG'] = round(tbl_PLD_original_USPSAG.apply(lambda row: 
                                                                  (row['FRT_USPSAG'] + (row['EXTRA_USPSAG_1'] + row['EXTRA_USPSAG_2'])/(1-Margin_on_USPS)) if row['FRT_USPSAG'] != '' else '', axis=1),2)                         
#tbl_PLD_original_USPSAG.to_csv('test.csv')


# USPSAP

In [1099]:
#Get the right client rate and the corresponding PLD
#get client rate and a copy of PLD
tbl_clientRates_18 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'USPSAP')]
tbl_PLD_original_USPSAP = tbl_BP.copy()

In [1100]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_USPSAP['BILLED_WEIGHT_LB'] = tbl_PLD_original_USPSAP.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_usps)) if (row['CUBIC_INCH'] > 1728) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

tbl_PLD_original_USPSAP['BILLED_WEIGHT_OZ'] = tbl_PLD_original_USPSAP.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_usps))*16 if (row['CUBIC_INCH'] > 1728) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)

tbl_PLD_original_USPSAP['BILLED_WEIGHT_LB'] = tbl_PLD_original_USPSAP.apply(lambda row:
                                                            0.5 if (row['BILLED_WEIGHT_OZ'] <= 8) else
                                                            (1 if (row['BILLED_WEIGHT_OZ'] > 8 and row['BILLED_WEIGHT_OZ'] <= 16) else             
                                                            (row['BILLED_WEIGHT_LB'])), axis=1)

#add Freight column to PLD
tbl_PLD_original_USPSAP_3 = tbl_PLD_original_USPSAP[(tbl_PLD_original_USPSAP['GIRTH_AND_L'] > 108) & (tbl_PLD_original_USPSAP['GIRTH_AND_L'] <= 130)]
tbl_PLD_original_USPSAP_none3 = tbl_PLD_original_USPSAP[(tbl_PLD_original_USPSAP['GIRTH_AND_L'] <= 108)]

#main case
tbl_PLD_original_USPSAP_none3['ZONE'] = tbl_PLD_original_USPSAP_none3['ZONE'].astype(str)
tbl_PLD_original_USPSAP_none3 = tbl_PLD_original_USPSAP_none3.merge(tbl_clientRates_18[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_USPSAP_none3 = tbl_PLD_original_USPSAP_none3.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_USPSAP_none3 = tbl_PLD_original_USPSAP_none3.rename(columns={'CLIENT_RATES': 'FRT_USPSAP'})

#3. for parcels that measure in combined length and girth more than 108 inches but not more than 130 inches, use oversized prices, 
#regardless of weight, based on the applicable zone.

tbl_oversize = tbl_clientRates_18[(tbl_clientRates_18['WEIGHT_LB'] == 9999)]
tbl_PLD_original_USPSAP_3['ZONE'] = tbl_PLD_original_USPSAP_3['ZONE'].astype(str)
tbl_PLD_original_USPSAP_3 = tbl_PLD_original_USPSAP_3.merge(tbl_oversize[["ZONE", "CLIENT_RATES"]], left_on=['ZONE'], right_on=['ZONE'], how='left')
tbl_PLD_original_USPSAP_3 = tbl_PLD_original_USPSAP_3.rename(columns={'CLIENT_RATES': 'FRT_USPSAP'})

#combine again
tbl_PLD_original_USPSAP = pd.concat([tbl_PLD_original_USPSAP_3, tbl_PLD_original_USPSAP_none3], axis=0)

#6. Parcels that exceed 22 inches but not greater than 30 inches in length, add $4 per package. 
#7. parcels that exceed 30 inches in length, add $7 per package. 




C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\1624375625.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_PLD_original_USPSAP_none3['ZONE'] = tbl_PLD_original_USPSAP_none3['ZONE'].astype(str)
C:\Users\sliu\AppData\Local\Temp\ipykernel_18976\1624375625.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_PLD_original_USPSAP_3['ZONE'] = tbl_PLD_original_USPSAP_3['ZONE'].astype(str)


In [1101]:

tbl_PLD_original_USPSAP['EXTRA_USPSAP_1'] = tbl_PLD_original_USPSAP.apply(lambda row:
                                                            4 if (row['L'] > 22 and row['L'] <= 30) else 
                                                           (7 if (row['L'] > 30) else
                                                            (0)), axis=1)


#8. parcels that exceed 2 cubic feet (3456 cubic inches), add $15 per package. 
tbl_PLD_original_USPSAP['EXTRA_USPSAP_2'] = tbl_PLD_original_USPSAP.apply(lambda row:
                                                            15 if (row['CUBIC_INCH'] > 3456) else (0), axis=1)

#Get Total
# note that for those with non-round up actual weight, if it is < 1, we still assign it the freight rate. 

#Get Total, revise the logic that if the freight rate is 0 or some rate is 0, the total will be 0. ups can handle everything. 
tbl_PLD_original_USPSAP['TOTAL_USPSAP'] = round(tbl_PLD_original_USPSAP.apply(lambda row: 
                                                                  (row['FRT_USPSAP'] + (row['EXTRA_USPSAP_1'] + row['EXTRA_USPSAP_2'])/(1-Margin_on_USPS)) if row['FRT_USPSAP'] != '' else '', axis=1),2)                         
#tbl_PLD_original_USPSAP.to_csv('test.csv')

# MI

#MIPLE (MI Parcel Select Lightweight Expedited): commercial & residential, oz

#MIPH (MI Parcel Select Heavyweight): commercial & residential, lb

# MIPLE (MI Parcel Select Lightweight Expedited)

In [1029]:
#select the rate to use, #Decide the part of PLD to rerate
tbl_clientRates_19 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'MIPLE')]
tbl_PLD_original_MIPLE = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] < 1) & (tbl_BP['L'] <= 26) & (tbl_BP['W'] <= 26) & (tbl_BP['H'] <= 26)].copy()

In [1030]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_MIPLE['BILLED_WEIGHT_OZ'] = tbl_PLD_original_MIPLE.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_mi))*16 if (row['CUBIC_INCH'] > 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)


#add Freight column to PLD
tbl_PLD_original_MIPLE['ZONE'] = tbl_PLD_original_MIPLE['ZONE'].astype(str)
tbl_PLD_original_MIPLE = tbl_PLD_original_MIPLE.merge(tbl_clientRates_19[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_MIPLE = tbl_PLD_original_MIPLE.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_MIPLE = tbl_PLD_original_MIPLE.rename(columns={'CLIENT_RATES': 'FRT_MIPLE'})

#add fuel surcharge column to PLD
tbl_PLD_original_MIPLE['FSC_MIPLE'] = tbl_PLD_original_MIPLE.apply(lambda row: row['FRT_MIPLE']*0.065, axis=1)

#add NQD Surcharge column to PLD

# A Non-standard piece fee of $4.00 will apply to any piece with a single side over 22" or with total dimensions over 2 ft.
tbl_PLD_original_MIPLE['NQD_MIPLE'] = tbl_PLD_original_MIPLE.apply(lambda row:
                                                            4 if any([row['L'] > 22, row['W'] > 22, row['H'] > 22, row['CUBIC_INCH'] > 3456]) else
                                                           (0), axis=1)

#Get Total
tbl_PLD_original_MIPLE['TOTAL_MIPLE'] = round(tbl_PLD_original_MIPLE.apply(lambda row: 
                                                                  np.nan if ((row['FRT_MIPLE'] == '') | (pd.isna(row['FRT_MIPLE'])))
                                                                  else
                                                                  ((row['FRT_MIPLE'] + row['FSC_MIPLE']) + (row['NQD_MIPLE'])/(1-Margin_on_MI)), axis=1),2)

#tbl_PLD_original_MIPLE.to_csv('test.csv')

# MIPH (MI Parcel Select Heavyweight)

In [1031]:
#select the rate to use, #Decide the part of PLD to rerate
tbl_clientRates_20 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'MIPH')]
tbl_PLD_original_MIPH = tbl_BP[(tbl_BP['L'] <= 26) & (tbl_BP['W'] <= 26) & (tbl_BP['H'] <= 26)].copy()

In [1032]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_MIPH['BILLED_WEIGHT_LB'] = tbl_PLD_original_MIPH.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_mi)) if (row['CUBIC_INCH'] > 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)



#add Freight column to PLD
tbl_PLD_original_MIPH['ZONE'] = tbl_PLD_original_MIPH['ZONE'].astype(str)
tbl_PLD_original_MIPH = tbl_PLD_original_MIPH.merge(tbl_clientRates_20[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_MIPH = tbl_PLD_original_MIPH.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_MIPH = tbl_PLD_original_MIPH.rename(columns={'CLIENT_RATES': 'FRT_MIPH'})

#add fuel surcharge column to PLD
tbl_PLD_original_MIPH['FSC_MIPH'] = tbl_PLD_original_MIPH.apply(lambda row: row['FRT_MIPH']*0.065, axis=1)


#add NQD Surcharge column to PLD

# A Non-standard piece fee of $4.00 will apply to any piece with a single side over 22" or with total dimensions over 2 ft.
tbl_PLD_original_MIPH['NQD_MIPH'] = tbl_PLD_original_MIPH.apply(lambda row:
                                                            4 if any([row['L'] > 22, row['W'] > 22, row['H'] > 22, row['CUBIC_INCH'] > 3456]) else
                                                           (0), axis=1)

#Get Total
tbl_PLD_original_MIPH['TOTAL_MIPH'] = round(tbl_PLD_original_MIPH.apply(lambda row: 
                                                                  np.nan if ((row['FRT_MIPH'] == '') | (pd.isna(row['FRT_MIPH'])))
                                                                  else
                                                                  ((row['FRT_MIPH'] + row['FSC_MIPH']) + (row['NQD_MIPH'])/(1-Margin_on_MI)), axis=1),2)

#tbl_PLD_original_MIPH.to_csv('test.csv')

# 1.  Model Output

# Specify the columns to keep in the model

In [1102]:
tbl_BP.columns

Index(['CustomerID', 'CustomerName', 'Facility', 'BarrettOrderNumber',
       'Reference', 'PoNumber', 'TRACKING_NUMBER', 'SHIP_DATE', 'OLD_SERVICE',
       'Shipper', 'ShipToName', 'Contact', 'City', 'State', 'ZIP_CODE',
       'Country', 'ZONE', 'Quantity', 'DIMENSIONS', 'ACTUAL_WEIGHT',
       'RESIDENTIAL_FLAG', 'L', 'W', 'H', 'GIRTH_AND_L', 'CUBIC_INCH',
       'ACTUAL_WEIGHT_LB', 'ACTUAL_WEIGHT_OZ'],
      dtype='object')

In [1103]:
# Make a copy of the DataFrame with only the specified columns
tbl_PLD_original_rerated = tbl_BP.copy()

# Specify the prioritized Services

In [1035]:
tbl_PLD_original_REDE = tbl_PLD_original_REDE.drop_duplicates()
tbl_PLD_original_RED = tbl_PLD_original_RED.drop_duplicates()
tbl_PLD_original_REDS = tbl_PLD_original_REDS.drop_duplicates()
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.drop_duplicates()
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.drop_duplicates()
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.drop_duplicates()
tbl_PLD_original_GRND = tbl_PLD_original_GRND.drop_duplicates()
tbl_PLD_original_GRES = tbl_PLD_original_GRES.drop_duplicates()
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.drop_duplicates()
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.drop_duplicates()
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.drop_duplicates()
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.drop_duplicates()
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.drop_duplicates()
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.drop_duplicates()
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.drop_duplicates()
tbl_PLD_original_USPSAG = tbl_PLD_original_USPSAG.drop_duplicates()
tbl_PLD_original_USPSAP = tbl_PLD_original_USPSAP.drop_duplicates()
tbl_PLD_original_MIPH = tbl_PLD_original_MIPH.drop_duplicates()
tbl_PLD_original_MIPLE = tbl_PLD_original_MIPLE.drop_duplicates()

In [1036]:
# List only the total rates of all carrier services. 
tbl_PLD_original_rerated['TOTAL_REDE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_REDE.set_index('TRACKING_NUMBER')['TOTAL_REDE'])
tbl_PLD_original_rerated['TOTAL_RED'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_RED.set_index('TRACKING_NUMBER')['TOTAL_RED'])
tbl_PLD_original_rerated['TOTAL_REDS'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_REDS.set_index('TRACKING_NUMBER')['TOTAL_REDS'])
tbl_PLD_original_rerated['TOTAL_2DAM'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_2DAM.set_index('TRACKING_NUMBER')['TOTAL_2DAM'])
tbl_PLD_original_rerated['TOTAL_BLUE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_BLUE.set_index('TRACKING_NUMBER')['TOTAL_BLUE'])
tbl_PLD_original_rerated['TOTAL_ORNG'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_ORNG.set_index('TRACKING_NUMBER')['TOTAL_ORNG'])
tbl_PLD_original_rerated['TOTAL_GRND'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_GRND.set_index('TRACKING_NUMBER')['TOTAL_GRND'])
tbl_PLD_original_rerated['TOTAL_GRES'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_GRES.set_index('TRACKING_NUMBER')['TOTAL_GRES'])
tbl_PLD_original_rerated['TOTAL_SRPT1'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT1.set_index('TRACKING_NUMBER')['TOTAL_SRPT1'])
tbl_PLD_original_rerated['TOTAL_SRPT'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT.set_index('TRACKING_NUMBER')['TOTAL_SRPT'])

tbl_PLD_original_rerated['TOTAL_DHLG'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG.set_index('TRACKING_NUMBER')['TOTAL_DHLG'])
tbl_PLD_original_rerated['TOTAL_DHLG1'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG1.set_index('TRACKING_NUMBER')['TOTAL_DHLG1'])
tbl_PLD_original_rerated['TOTAL_DHLE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE.set_index('TRACKING_NUMBER')['TOTAL_DHLE'])
tbl_PLD_original_rerated['TOTAL_DHLE1'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE1.set_index('TRACKING_NUMBER')['TOTAL_DHLE1'])
tbl_PLD_original_rerated['TOTAL_DHLEM'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLEM.set_index('TRACKING_NUMBER')['TOTAL_DHLEM'])

tbl_PLD_original_rerated['TOTAL_USPSAG'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAG.set_index('TRACKING_NUMBER')['TOTAL_USPSAG'])
tbl_PLD_original_rerated['TOTAL_USPSAP'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAP.set_index('TRACKING_NUMBER')['TOTAL_USPSAP'])

tbl_PLD_original_rerated['TOTAL_MIPLE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_MIPLE.set_index('TRACKING_NUMBER')['TOTAL_MIPLE'])
tbl_PLD_original_rerated['TOTAL_MIPH'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_MIPH.set_index('TRACKING_NUMBER')['TOTAL_MIPH'])


# Base Model

In [1106]:
# Base model output. Make sure to check the obnormal rates due to obnormal dim, etc. 
tbl_PLD_original_rerated.to_csv('test.csv')

# ******End******